In [4]:
#%matplotlib notebook
%matplotlib qt

#This notebook is a testbed for importing PEAC Center USAPI
#raifnall data using pandas and doing some quick analysis

import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as cols
import matplotlib.cm as cm

from mpl_toolkits.basemap import Basemap, shiftgrid

import numpy as np
import numpy.ma as ma

from datetime import date
import datetime

import calendar

from netCDF4 import Dataset

from scipy import signal, linalg, stats

from pycurrents.codas import to_day, to_date
from pycurrents.plot.mpltools import dday_to_mpl

from pycurrents.system import Bunch
from pycurrents.num import eof
from pycurrents.num import rangeslice

import pickle

from scipy.special import comb

In [5]:
datadir = '/home/alejandro/Desktop/Data/'

In [6]:
# NCEP UWND, VWND ahd HGT all go from Jan 1948 to present and are on a 2.5 deg grid
# the grid goes from 0.0E to 357.5E and 90N to 90S
uwnd_data = Dataset(datadir + 'uwnd.mon.mean.nc')
vwnd_data = Dataset(datadir + 'vwnd.mon.mean.nc')
hgt_data = Dataset(datadir + 'hgt.mon.mean.nc')

# read lats,lons
ncep_latitudes = uwnd_data.variables['lat'][:]
ncep_longitudes = uwnd_data.variables['lon'][:]

#read in ncep levels
# these levels are
#1000,925,850,700,600,500,400,300,250,200,150,100,70,50,30,20,10
ncep_levels = uwnd_data.variables['level'][:]

#Choose pressure level now, to save obn memory
pressure_selection = ncep_levels[:] == 850


#time in hours since 1800-01-01 00:00:0.0

ncep_time = uwnd_data.variables['time'][:]

# get  wind data.
uin = np.squeeze(uwnd_data.variables['uwnd'][:, pressure_selection, :, :])
vin = np.squeeze(vwnd_data.variables['vwnd'][:, pressure_selection, :, :])
hgtin = np.squeeze(hgt_data.variables['hgt'][:, pressure_selection, :, :])

uwnd_data.close()
vwnd_data.close()
hgt_data.close()

# Let's just convert it to days:
ncep_dday_1800 = ncep_time / 24

#----------------------------------------------------------------
#GPCP data goes from Jan 1979 to present and is on a 2.5 deg grid
#same grid as NCEP
precip_data = Dataset(datadir + 'precip.mon.mean.nc')

#time in hours since 1800-01-01 00:00:0.0
precip_time = precip_data.variables['time'][:]

# read lats,lons
precip_latitudes = precip_data.variables['lat'][:]
precip_longitudes = precip_data.variables['lon'][:]

# get precip data.
precip_in = precip_data.variables['precip'][:, :, :]

precip_data.close()

# GPCP time is in days since jan 1 1800
precip_dday_1800 = precip_time

#----------------------------------------------------------------
#PREC data goes from Jan 1948 to present and is on a 2.5 deg grid
#same grid as NCEP
PREC_precip_data = Dataset(datadir + 'PREC.precip.mon.anom.nc')

#time in hours since 1800-01-01 00:00:0.0
PREC_precip_time = PREC_precip_data.variables['time'][:]

# read lats,lons
PREC_precip_latitudes = PREC_precip_data.variables['lat'][:]
PREC_precip_longitudes = PREC_precip_data.variables['lon'][:]

# get precip data.
PREC_precip_in = PREC_precip_data.variables['precip'][:, :, :]

PREC_precip_data.close()

# PREC time is in hours since jan 1 1800
PREC_precip_dday_1800 = PREC_precip_time /24

#----------------------------------------------------------------
#ERSST v4 data goes from Jan 1854 to present and is on a 2 deg grid
#88.0N - 88.0S, 0.0E - 358.0E.
ersst_data = Dataset(datadir + 'sst.mnmean.v4.nc')

# read lats,lons
ersst_latitudes = ersst_data.variables['lat'][:]
ersst_longitudes = ersst_data.variables['lon'][:]

#time in hours since 1800-01-01 00:00:0.0
ersst_time = ersst_data.variables['time'][:]

# get  wind data.
ersst_in = ersst_data.variables['sst'][:, :, :]

ersst_data.close()

# ERSST time is in days since jan 1 1800
ersst_dday_1800 = ersst_time

In [7]:
def extend(dday_1800, dat):
    """
    Return dday, dat extended to fill out the last year.
    
    This function was modifyed to take in only one data variable.
    """
    n_orig = len(dday_1800)
    ymdhms = to_date(1800, dday_1800)
    nmissing = 12 - ymdhms[-1, 1]
    nmonths = n_orig + nmissing
    ymdhms_new = np.zeros((nmonths, 6), dtype=np.uint16)
    ymdhms_new[:n_orig, :] = ymdhms
    ymdhms_new[n_orig:, 0] = ymdhms[-1, 0]
    # Fill in the remaining months:
    ymdhms_new[n_orig:, 1] = np.arange(ymdhms[-1, 1] + 1, 13)
    ymdhms_new[n_orig:, 2] = 1
    dday_new = to_day(1800, ymdhms_new)
    
    shape_new = (nmonths, dat.shape[1], dat.shape[2])
    ## EF: modify to work with masked array or ndarray
    if np.ma.isMA(dat):
        dat_new = np.ma.masked_all(shape_new, float)
    else:    
        dat_new = np.nan + np.zeros(shape_new, float)
    
    dat_new[:n_orig, :, :] = dat
    
    #This Function has been modifyed to also return the date of the last valid data and its ymdhms index
    
    last_valid_date = to_date(1800,dday_1800[-1])
    last_valid_index = n_orig -1
   
    return Bunch(dday_1800=dday_new,
                 ymdhms=ymdhms_new,
                 dat=dat_new,
                 last_valid_date=last_valid_date,
                 last_valid_index=last_valid_index)

def cal_climatology_and_anomaly(data,ymdhms,last_valid_date,last_valid_index,latitudes,longitudes,start_year, end_year):
                           
    #Select the years for climatology from the new ymdhms_padded
    # All the monthly data for the years you want to calculate the climatology
    
    clim_selection = ((ymdhms[:, 0] >= start_year) & (ymdhms[:, 0] <= end_year))
    
    # EF: Trim everything right at the start.
    data = data[clim_selection]
    ymdhms = ymdhms[clim_selection]
    
    #Calculate the total numbers of years in the climatology
    length_in_years = ymdhms[-1, 0] - ymdhms[0, 0] + 1
    
    # EF: ensure we have masked arrays and no nans
    data = np.ma.masked_invalid(data)
    
    #reshape the matrix so that is has dimensions of [years, months, lat, lon]
    reshaped_data = np.reshape(data, (length_in_years, 12, 
                                      len(latitudes), 
                                      len(longitudes)))
    #Calculate the climatology by using nanmeans, along the yeas axis
    #climatology = np.nanmean(reshaped_data, axis=0)
    
    climatology = reshaped_data.mean(axis=0)
    anomaly = reshaped_data - climatology
    anomaly = anomaly.reshape(data.shape)
    
    if end_year < last_valid_date[0]:
        last_valid_date_new = ymdhms[-1,:]
        last_valid_index_new = -1
    else:
        last_valid_date_new = last_valid_date
        last_valid_index_new = np.where((ymdhms[:,0]== last_valid_date[0]) &
                                        (ymdhms[:,1]== last_valid_date[1]) &
                                        (ymdhms[:,2]== last_valid_date[2]))[0][0]
    
    return Bunch(climatology=climatology,
                 anomaly=anomaly, 
                 ymdhms=ymdhms,
                 last_valid_date = last_valid_date_new,
                 last_valid_index = last_valid_index_new)

def seasonal_anomaly(m_anom,last_valid_index):
    s_anom = np.ma.zeros(m_anom.shape, float)
    orig_mask = np.ma.getmaskarray(m_anom)
    
    if last_valid_index == -1:
        s_anom[1:-1] = (m_anom[:-2] + m_anom[1:-1] + m_anom[2:]) / 3
    
        s_anom[0] = (m_anom[0] + m_anom[1]) / 2
        s_anom[-1] = (m_anom[-2] + m_anom[-1]) / 2
    
    else:
        #s_anom[1:last_valid_index-1,:,:] = (m_anom[:last_valid_index-2,:,:] + 
        #                                    m_anom[1:last_valid_index-1,:,:] + 
        #                                    m_anom[2:last_valid_index,:,:]) / 3
        s_anom[1:last_valid_index,:,:] = (m_anom[:last_valid_index-1,:,:] + 
                                          m_anom[1:last_valid_index,:,:] + 
                                          m_anom[2:last_valid_index+1,:,:]) / 3
    
        s_anom[0,:,:] = (m_anom[0,:,:] + m_anom[1,:,:]) / 2
        s_anom[last_valid_index,:,:] = (m_anom[last_valid_index-1,:,:] + 
                                        m_anom[last_valid_index,:,:]) / 2
        
    s_anom_out = np.ma.array(s_anom, mask=orig_mask)
    
    return Bunch(s_anom = s_anom_out, 
                 last_valid_index = last_valid_index)

def seasonal_anomaly2(m_anom, nmin=2):
    newshape = [3] + list(m_anom.shape)
    newshape[1] += 2
    accum = np.ma.zeros(newshape, dtype=m_anom.dtype)
    accum[:] = np.ma.masked
    accum[0, :-2] = m_anom[:]
    accum[1, 1:-1] = m_anom[:]
    accum[2, 2:] = m_anom[:]
    out = accum.mean(axis=0)[1:-1]
    out = np.ma.masked_where(accum[:, 1:-1].count(axis=0) < nmin, out, copy=False)
    return Bunch(s_anom = out)

def running_sum(m_anom, window, nmin = 2):
    nmin = (window+1)/2
    chop = (window -1)/2

    newshape = [window] + list(m_anom.shape)
    newshape[1] += window -1
    accum = np.ma.zeros(newshape, dtype=m_anom.dtype)
    accum[:] = np.ma.masked
    for i in range(window):
        end = -window+i+1
        if end == 0:
            accum[i, i:] = m_anom[:]
        else:
            accum[i, i:end] = m_anom[:]
    out = accum.mean(axis=0)[chop:-chop]
    out = np.ma.masked_where(accum[:, chop:-chop].count(axis=0) < nmin, out, copy=False)
    return Bunch(s_anom = out)

def running_sum_2(m_anom, window, nmin = 2):
    
    if window % 2 == 0:
        nmin = (window)/2
        #nmin = 1
        chop = (window-1)

        newshape = [window] + list(m_anom.shape)
        newshape[1] += window -1
        accum = np.ma.zeros(newshape, dtype=m_anom.dtype)
        accum[:] = np.ma.masked
        for i in range(window):
            end = -window+i+1
            if end == 0:
                accum[i, i:] = m_anom[:]
            else:
                accum[i, i:end] = m_anom[:]
        if window != 2:
            start=window/2
            stop = (window/2) -1
            out = accum.mean(axis=0)[start:-stop]
            out = np.ma.masked_where(accum[:,start:-stop].count(axis=0) < nmin, out, copy=False)
        else:
            out = accum.mean(axis=0)[1:]
            out = np.ma.masked_where(accum[:,1:].count(axis=0) < nmin, out, copy=False)
        
    else:
        nmin = (window+1)/2
        chop = (window -1)/2

        newshape = [window] + list(m_anom.shape)
        newshape[1] += window -1
        accum = np.ma.zeros(newshape, dtype=m_anom.dtype)
        accum[:] = np.ma.masked
        for i in range(window):
            end = -window+i+1
            if end == 0:
                accum[i, i:] = m_anom[:]
            else:
                accum[i, i:end] = m_anom[:]
        out = accum.mean(axis=0)[chop:-chop]
        out = np.ma.masked_where(accum[:, chop:-chop].count(axis=0) < nmin, out, copy=False)
        
    return Bunch(s_anom = out)

In [8]:
#fill out the data for the missing months at the end of the last year of the dataset
precip = extend(precip_dday_1800,precip_in)
PREC_precip = extend(PREC_precip_dday_1800,PREC_precip_in)

uwnd = extend(ncep_dday_1800,uin)
vwnd = extend(ncep_dday_1800,vin)
hgt = extend(ncep_dday_1800,hgtin)

sst = extend(ersst_dday_1800,ersst_in)

In [9]:
mpldays_precip = dday_to_mpl(1800, precip_dday_1800)
mpldays_ncep = dday_to_mpl(1800, ncep_dday_1800)

In [10]:
#Here we calculate the anomaly
precip_ca = cal_climatology_and_anomaly(precip.dat,precip.ymdhms,
                                        precip.last_valid_date, precip.last_valid_index,
                                        precip_latitudes,
                                        precip_longitudes,1979,2015)

PREC_precip_ca = cal_climatology_and_anomaly(PREC_precip.dat,PREC_precip.ymdhms,
                                             PREC_precip.last_valid_date, PREC_precip.last_valid_index,
                                             PREC_precip_latitudes,
                                             PREC_precip_longitudes,1979,2015)

uwnd_ca = cal_climatology_and_anomaly(uwnd.dat,uwnd.ymdhms,
                                      uwnd.last_valid_date,uwnd.last_valid_index,
                                      ncep_latitudes,
                                      ncep_longitudes,1979,2015)

vwnd_ca = cal_climatology_and_anomaly(vwnd.dat,vwnd.ymdhms,
                                      vwnd.last_valid_date,vwnd.last_valid_index,
                                      ncep_latitudes,
                                      ncep_longitudes,1979,2015)

hgt_ca = cal_climatology_and_anomaly(hgt.dat,hgt.ymdhms,
                                     hgt.last_valid_date,hgt.last_valid_index,
                                     ncep_latitudes,
                                     ncep_longitudes,1979,2015)

sst_ca = cal_climatology_and_anomaly(sst.dat,sst.ymdhms,
                                     sst.last_valid_date,sst.last_valid_index,
                                     ersst_latitudes,
                                     ersst_longitudes,1979,2015)

precip_seasonal_anom = running_sum_2(precip_ca.anomaly,6, nmin=4)
PREC_precip_seasonal_anom = running_sum_2(PREC_precip_ca.anomaly,6, nmin=4)

uwnd_seasonal_anom = running_sum_2(uwnd_ca.anomaly,6, nmin=4)
vwnd_seasonal_anom = running_sum_2(vwnd_ca.anomaly,6, nmin=4)
hgt_seasonal_anom = running_sum_2(hgt_ca.anomaly,6, nmin=4)

sst_seasonal_anom = running_sum_2(sst_ca.anomaly,6, nmin=4)

/home/alejandro/anaconda3/lib/python3.5/site-packages/numpy/ma/core.py:3114: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  dout = self.data[indx]
/home/alejandro/anaconda3/lib/python3.5/site-packages/numpy/ma/core.py:3169: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  dout._mask = _mask[indx]


# Calculate the Composite maps

## Here i use the years chosen by hand for Nov-May SPI and cool ONI conditions

## The years are 1981, 1984, 1999, 2001, 2009, 2013

In [11]:
year_list = [1984, 2001, 2006, 2009, 2013]
non_dry_year_list = [1985, 1986, 1996, 1997, 2014]
la_nina_list =  [1989, 1999, 2000, 2008, 2011]

In [12]:
gpcp_season_members_verydry = np.zeros(12)

sst_composite_members_verydry = np.empty([50,24, len(ersst_latitudes), len(ersst_longitudes)])
sst_composite_members_verydry[:] = np.NAN

precip_composite_members_verydry = np.empty([50,24, len(precip_latitudes), len(precip_longitudes)])
precip_composite_members_verydry[:] = np.NAN

uwnd_composite_members_verydry = np.empty([50,24,len(ncep_latitudes), len(ncep_longitudes)])
uwnd_composite_members_verydry[:] = np.NAN
vwnd_composite_members_verydry = np.empty([50,24,len(ncep_latitudes), len(ncep_longitudes)])
vwnd_composite_members_verydry[:] = np.NAN
hgt_composite_members_verydry = np.empty([50,24,len(ncep_latitudes), len(ncep_longitudes)])
hgt_composite_members_verydry[:] = np.NAN

gpcp_season_members_nondry = np.zeros(12)

sst_composite_members_nondry = np.empty([50,24, len(ersst_latitudes), len(ersst_longitudes)])
sst_composite_members_nondry[:] = np.NAN

precip_composite_members_nondry = np.empty([50,24, len(precip_latitudes), len(precip_longitudes)])
precip_composite_members_nondry[:] = np.NAN

uwnd_composite_members_nondry = np.empty([50,24,len(ncep_latitudes), len(ncep_longitudes)])
uwnd_composite_members_nondry[:] = np.NAN
vwnd_composite_members_nondry = np.empty([50,24,len(ncep_latitudes), len(ncep_longitudes)])
vwnd_composite_members_nondry[:] = np.NAN
hgt_composite_members_nondry = np.empty([50,24,len(ncep_latitudes), len(ncep_longitudes)])
hgt_composite_members_nondry[:] = np.NAN

gpcp_season_members_lanina = np.zeros(12)

sst_composite_members_lanina = np.empty([50,24, len(ersst_latitudes), len(ersst_longitudes)])
sst_composite_members_lanina[:] = np.NAN

precip_composite_members_lanina = np.empty([50,24, len(precip_latitudes), len(precip_longitudes)])
precip_composite_members_lanina[:] = np.NAN

uwnd_composite_members_lanina = np.empty([50,24,len(ncep_latitudes), len(ncep_longitudes)])
uwnd_composite_members_lanina[:] = np.NAN
vwnd_composite_members_lanina = np.empty([50,24,len(ncep_latitudes), len(ncep_longitudes)])
vwnd_composite_members_lanina[:] = np.NAN
hgt_composite_members_lanina = np.empty([50,24,len(ncep_latitudes), len(ncep_longitudes)])
hgt_composite_members_lanina[:] = np.NAN

In [13]:
gpcp_season_members_verydry = np.zeros(24)
gpcp_season_members_nondry = np.zeros(24)
gpcp_season_members_lanina = np.zeros(24)

year_range = range(1966,2016)
short_year_range = range(1979,2016)
for year in short_year_range:
    for m in range(1,13):
        
        if year in year_list:
            
            #print(year,year_range.index(year),m,cond_verydry_cool[year_range.index(year),m-1])
            
            sst_selection = ((sst_ca.ymdhms[:, 0] == year-1) & (sst_ca.ymdhms[:, 1] == m))
            sst_selection = np.nonzero(sst_selection)[0][0]
            sst_composite_members_verydry[year_range.index(year),m-1,:,:] = sst_seasonal_anom.s_anom[sst_selection]

            sst_selection = ((sst_ca.ymdhms[:, 0] == year) & (sst_ca.ymdhms[:, 1] == m))
            sst_selection = np.nonzero(sst_selection)[0][0]
            sst_composite_members_verydry[year_range.index(year),m-1+12,:,:] = sst_seasonal_anom.s_anom[sst_selection]

            selection = ((uwnd_ca.ymdhms[:, 0] == year-1) & (uwnd_ca.ymdhms[:, 1] == m))
            selection = np.nonzero(selection)[0][0]
            uwnd_composite_members_verydry[year_range.index(year),m-1,:,:] = uwnd_seasonal_anom.s_anom[selection]
            vwnd_composite_members_verydry[year_range.index(year),m-1,:,:] = vwnd_seasonal_anom.s_anom[selection]
            hgt_composite_members_verydry[year_range.index(year),m-1,:,:] = hgt_seasonal_anom.s_anom[selection]
            
            selection = ((uwnd_ca.ymdhms[:, 0] == year) & (uwnd_ca.ymdhms[:, 1] == m))
            selection = np.nonzero(selection)[0][0]
            uwnd_composite_members_verydry[year_range.index(year),m-1+12,:,:] = uwnd_seasonal_anom.s_anom[selection]
            vwnd_composite_members_verydry[year_range.index(year),m-1+12,:,:] = vwnd_seasonal_anom.s_anom[selection]
            hgt_composite_members_verydry[year_range.index(year),m-1+12,:,:] = hgt_seasonal_anom.s_anom[selection]

            precip_selection = ((precip_ca.ymdhms[:, 0] == year-1) & (precip_ca.ymdhms[:, 1] == m))
            precip_selection = np.nonzero(precip_selection)[0][0]
            precip_composite_members_verydry[year_range.index(year),m-1,:,:] = precip_seasonal_anom.s_anom[precip_selection]
            
            precip_selection = ((precip_ca.ymdhms[:, 0] == year) & (precip_ca.ymdhms[:, 1] == m))
            precip_selection = np.nonzero(precip_selection)[0][0]
            precip_composite_members_verydry[year_range.index(year),m-1+12,:,:] = precip_seasonal_anom.s_anom[precip_selection]

            gpcp_season_members_verydry[m-1] = gpcp_season_members_verydry[m-1] +1
            gpcp_season_members_verydry[m-1+12] = gpcp_season_members_verydry[m-1+12] +1

        if year in non_dry_year_list:   


            #print(year,year_range.index(year),m,cond_verydry_cool[year_range.index(year),m-1])

            sst_selection = ((sst_ca.ymdhms[:, 0] == year-1) & (sst_ca.ymdhms[:, 1] == m))
            sst_selection = np.nonzero(sst_selection)[0][0]
            sst_composite_members_nondry[year_range.index(year),m-1,:,:] = sst_seasonal_anom.s_anom[sst_selection]
            
            sst_selection = ((sst_ca.ymdhms[:, 0] == year) & (sst_ca.ymdhms[:, 1] == m))
            sst_selection = np.nonzero(sst_selection)[0][0]
            sst_composite_members_nondry[year_range.index(year),m-1+12,:,:] = sst_seasonal_anom.s_anom[sst_selection]

            selection = ((uwnd_ca.ymdhms[:, 0] == year-1) & (uwnd_ca.ymdhms[:, 1] == m))
            selection = np.nonzero(selection)[0][0]
            uwnd_composite_members_nondry[year_range.index(year),m-1,:,:] = uwnd_seasonal_anom.s_anom[selection]
            vwnd_composite_members_nondry[year_range.index(year),m-1,:,:] = vwnd_seasonal_anom.s_anom[selection]
            hgt_composite_members_nondry[year_range.index(year),m-1,:,:] = hgt_seasonal_anom.s_anom[selection]
            
            selection = ((uwnd_ca.ymdhms[:, 0] == year) & (uwnd_ca.ymdhms[:, 1] == m))
            selection = np.nonzero(selection)[0][0]
            uwnd_composite_members_nondry[year_range.index(year),m-1+12,:,:] = uwnd_seasonal_anom.s_anom[selection]
            vwnd_composite_members_nondry[year_range.index(year),m-1+12,:,:] = vwnd_seasonal_anom.s_anom[selection]
            hgt_composite_members_nondry[year_range.index(year),m-1+12,:,:] = hgt_seasonal_anom.s_anom[selection]

            precip_selection = ((precip_ca.ymdhms[:, 0] == year-1) & (precip_ca.ymdhms[:, 1] == m))
            precip_selection = np.nonzero(precip_selection)[0][0]
            precip_composite_members_nondry[year_range.index(year),m-1,:,:] = precip_seasonal_anom.s_anom[precip_selection]
            
            precip_selection = ((precip_ca.ymdhms[:, 0] == year) & (precip_ca.ymdhms[:, 1] == m))
            precip_selection = np.nonzero(precip_selection)[0][0]
            precip_composite_members_nondry[year_range.index(year),m-1+12,:,:] = precip_seasonal_anom.s_anom[precip_selection]

            gpcp_season_members_nondry[m-1] = gpcp_season_members_nondry[m-1] +1
            gpcp_season_members_nondry[m-1+12] = gpcp_season_members_nondry[m-1+12] +1
            
        if year in la_nina_list:   

            #print(year,year_range.index(year),m,cond_verydry_cool[year_range.index(year),m-1])

            sst_selection = ((sst_ca.ymdhms[:, 0] == year-1) & (sst_ca.ymdhms[:, 1] == m))
            sst_selection = np.nonzero(sst_selection)[0][0]
            sst_composite_members_lanina[year_range.index(year),m-1,:,:] = sst_seasonal_anom.s_anom[sst_selection]
            
            sst_selection = ((sst_ca.ymdhms[:, 0] == year) & (sst_ca.ymdhms[:, 1] == m))
            sst_selection = np.nonzero(sst_selection)[0][0]
            sst_composite_members_lanina[year_range.index(year),m-1+12,:,:] = sst_seasonal_anom.s_anom[sst_selection]

            selection = ((uwnd_ca.ymdhms[:, 0] == year-1) & (uwnd_ca.ymdhms[:, 1] == m))
            selection = np.nonzero(selection)[0][0]
            uwnd_composite_members_lanina[year_range.index(year),m-1,:,:] = uwnd_seasonal_anom.s_anom[selection]
            vwnd_composite_members_lanina[year_range.index(year),m-1,:,:] = vwnd_seasonal_anom.s_anom[selection]
            hgt_composite_members_lanina[year_range.index(year),m-1,:,:] = hgt_seasonal_anom.s_anom[selection]
            
            selection = ((uwnd_ca.ymdhms[:, 0] == year) & (uwnd_ca.ymdhms[:, 1] == m))
            selection = np.nonzero(selection)[0][0]
            uwnd_composite_members_lanina[year_range.index(year),m-1+12,:,:] = uwnd_seasonal_anom.s_anom[selection]
            vwnd_composite_members_lanina[year_range.index(year),m-1+12,:,:] = vwnd_seasonal_anom.s_anom[selection]
            hgt_composite_members_lanina[year_range.index(year),m-1+12,:,:] = hgt_seasonal_anom.s_anom[selection]

            precip_selection = ((precip_ca.ymdhms[:, 0] == year-1) & (precip_ca.ymdhms[:, 1] == m))
            precip_selection = np.nonzero(precip_selection)[0][0]
            precip_composite_members_lanina[year_range.index(year),m-1,:,:] = precip_seasonal_anom.s_anom[precip_selection]
            
            precip_selection = ((precip_ca.ymdhms[:, 0] == year) & (precip_ca.ymdhms[:, 1] == m))
            precip_selection = np.nonzero(precip_selection)[0][0]
            precip_composite_members_lanina[year_range.index(year),m-1+12,:,:] = precip_seasonal_anom.s_anom[precip_selection]

            gpcp_season_members_lanina[m-1] = gpcp_season_members_lanina[m-1] +1
            gpcp_season_members_lanina[m-1+12] = gpcp_season_members_lanina[m-1+12] +1

In [14]:
precip_composite_verydry = np.nanmean(precip_composite_members_verydry, axis=0)
sst_composite_verydry = np.nanmean(sst_composite_members_verydry, axis=0)
uwnd_composite_verydry = np.nanmean(uwnd_composite_members_verydry, axis=0)
vwnd_composite_verydry = np.nanmean(vwnd_composite_members_verydry, axis=0)
hgt_composite_verydry = np.nanmean(hgt_composite_members_verydry, axis=0)

In [15]:
precip_composite_nondry = np.nanmean(precip_composite_members_nondry, axis=0)
sst_composite_nondry = np.nanmean(sst_composite_members_nondry, axis=0)
uwnd_composite_nondry = np.nanmean(uwnd_composite_members_nondry, axis=0)
vwnd_composite_nondry = np.nanmean(vwnd_composite_members_nondry, axis=0)
hgt_composite_nondry = np.nanmean(hgt_composite_members_nondry, axis=0)

In [16]:
precip_composite_lanina = np.nanmean(precip_composite_members_lanina, axis=0)
sst_composite_lanina = np.nanmean(sst_composite_members_lanina, axis=0)
uwnd_composite_lanina = np.nanmean(uwnd_composite_members_lanina, axis=0)
vwnd_composite_lanina = np.nanmean(vwnd_composite_members_lanina, axis=0)
hgt_composite_lanina = np.nanmean(hgt_composite_members_lanina, axis=0)

# Here I define the plotting function and plot the figures

In [17]:
def plotting_function(color_field, contour_field, u_wnd, v_wnd, member_counter,  
                      color_field_name, case_name, target_dir, *argv, save = 'yes', field = 'sst'):    
    
    #here basemap is producing a strange behaviour
    m = Basemap(projection='merc', llcrnrlat=-50.1, urcrnrlat=50.01,
                        llcrnrlon=80, urcrnrlon=300, lat_ts=0, resolution='c')

    with open('mapcache.pk', mode='wb') as f:
        pickle.dump(m, f)

    ## Tweak the subplot specifications.  

    subplotparams = dict(left=0.03, right=0.88,
                         bottom=0.015, top=0.94,
                         wspace=0.05, hspace=0.001)


    fig, axs = plt.subplots(3,3, #sharex=True,
                        figsize=(13, 7.8),
                        gridspec_kw=subplotparams,
                       )

    mm = range(9,18)

    for mon, pax in zip(mm, axs.flat):

        with open('mapcache.pk', 'rb') as f:
            m = pickle.load(f)
        m.ax = pax
        
        if field == 'precip':
            #reverse the coolwarm palatte to make blue rainy and red dry
            cmap = plt.get_cmap('BrBG')
            bounds = [-2.5,-2, -1.5,-1.25, -1, -0.75, -0.5, -0.25, 0, 0.25, 0.5, 0.75, 1, 1.25, 1.5, 2, 2.5]


            x, y = m(*np.meshgrid(PREC_precip_longitudes, PREC_precip_latitudes))
            im = m.contourf(x, y, color_field[mon,:,:], levels=bounds, cmap=cmap,
                            extend='both')

        elif field == 'sst':
                #reverse the coolwarm palatte to make blue rainy and red dry
                cmap = cm.coolwarm
                bounds = [-2, -1.5,-1.25, -1, -0.75, -0.5, -0.25, 0, 0.25, 0.5, 0.75, 1, 1.25, 1.5, 2]

                x, y = m(*np.meshgrid(ersst_longitudes, ersst_latitudes))
                im = m.contourf(x, y, color_field[mon,:,:], levels=bounds, cmap=cmap,
                                extend='both')
        #------------------------------------------------

        x_hgt, y_hgt = m(*np.meshgrid(ncep_longitudes, ncep_latitudes))
        #hgt_bounds = np.arange(-40,40,1)
        hgt_bounds = [-80, -70,-60, -50, -40, 
                      -30, -20, -10,-8, -6, 
                      -4, -2, -1,0, 1, 2, 4, 
                      6, 8, 10, 20, 30,
                      40, 50, 60, 70, 80]
        hgt_bound_0 = [0]
        im_hgt = m.contour(x_hgt, y_hgt, contour_field[mon,:,:], levels=hgt_bounds,
                           linewidths=0.5, colors='k')
        im_hgt = m.contour(x_hgt, y_hgt, contour_field[mon,:,:], levels=hgt_bound_0,
                       linewidths=0.9, colors='k')

        # transform vectors to projection grid.
        uproj, vproj, xx, yy = m.rotate_vector(u_wnd[mon,:,:], 
                                               v_wnd[mon,:,:], 
                                               ncep_longitudes, 
                                               ncep_latitudes,
                                               returnxy=True)
        # now plot every other vector
        Q = m.quiver(xx[::2,::2], yy[::2,::2], 
                     uproj[::2,::2], vproj[::2,::2],
                     scale=20, scale_units='inches')

        m.drawcoastlines()
        
        if box_coords in argv:
            x1,y1 = m(box_coords[2], box_coords[1])
            x2,y2 = m(box_coords[2], box_coords[0])
            x3,y3 = m(box_coords[3], box_coords[0])
            x4,y4 = m(box_coords[3], box_coords[1])
            
            p = Polygon([(x1,y1), (x2,y2), (x3,y3), (x4,y4)], 
                        edgecolor='b', facecolor = 'none', linewidth = 2)
            m.ax.add_patch(p)

        
        parallels = np.arange(-90, 90, 30)
        meridians = np.arange(-180, 180, 60)
        if pax in axs.flat[::3]:
            m.drawparallels(parallels, labels = [1, 0, 0, 1], fontsize=8)
        else:
            m.drawparallels(parallels, labels = [0, 0, 0, 0], fontsize=8)

        if pax in axs.flat[:6]:
            m.drawmeridians(meridians, labels = [0, 0, 0, 0], fontsize=8)
        if pax in axs.flat[6:]:
            m.drawmeridians(meridians, labels = [1, 0, 0, 1], fontsize=8)

        #m.drawparallels(parallels, labels = [1, 0, 0, 1], fontsize=8)
        #m.drawmeridians(meridians, labels = [1, 0, 0, 1], fontsize=8)

        #pax.set_title(color_field_name+" Seasonal Anom " +season_list[mon])
        pax.set_title(season_list[mon])

    #cax, kw = mpl.colorbar.make_axes([ax for ax in axs.flat])
    ##  This method of making an Axes for the cbar interacts very badly with
    ##  basemap, so just make one manually.  This also gives more control, so
    ##  it looks nicer.
    left = subplotparams['right'] + 0.02
    bottom = subplotparams['bottom'] + 0.05
    width = 0.015
    height = subplotparams['top'] - subplotparams['bottom'] - 0.1

    cax = fig.add_axes([left, bottom, width, height])
    cb = plt.colorbar(im, cax=cax)
    
    if field == 'precip':
        cb.set_label('Precip Anomaly mm/day')
    elif field == 'sst':
        cb.set_label('SST Anomaly C')
    

    ## EF: The quiverkey needs to be made using an axes in which a quiver is
    ##     drawn so that it has the right transform information.  Otherwise
    ##     it won't get the length right.
    qkx = left + (1 - left) / 4
    qky = bottom + height + 0.025
    qk = pax.quiverkey(Q, qkx, qky, 10, '10 m/s', 
                       coordinates='figure',
                       labelpos='N',
                       labelsep=0.07, 
                       fontproperties=dict(size='small'),
                      )
    
    if field == 'precip':
        plt.suptitle(case_name+' '+color_field_name+' Precip 850HGT WND seasonal anomaly')
    elif field == 'sst':
        plt.suptitle(case_name+' '+color_field_name+' SST 850HGT WND seasonal anomaly')
    
    
    if save == 'yes':
        filename = target_dir+str(case_name)+"_"+color_field_name+field+'_850HGT_WND_seasonal_anomaly'
        fig.savefig(filename+".pdf")
        fig.savefig(filename+".png")
        #fig.savefig("VERY_DRY_COMPOSITE_SST_850HGT_WND_seasonal_anomaly_%04d-%02d.pdf" % (year, mon))
        plt.close()# no need for dpi

In [19]:
box_coords =[]
season_list = ['Nov-Apr(-1)' , 'Dec-May(-1)' , 'Jan-Jun(-1)',
               'Feb-Jul(-1)' , 'Mar-Aug(-1)' , 'Apr-Sep(-1)',
               'May-Oct(-1)' , 'Jun-Nov(-1)' , 'Jul-Dec(-1)',
               'Aug-Jan(-1)' , 'Sep-Feb(-1)' , 'Oct-Mar(-1)',
               'Nov-Apr(0)' , 'Dec-May(0)' , 'Jan-Jun(0)',
               'Feb-Jul(0)' , 'Mar-Aug(0)' , 'Apr-Sep(0)',
               'May-Oct(0)' , 'Jun-Nov(0)' , 'Jul-Dec(0)',
               'Aug-Jan(0)' , 'Sep-Feb(0)' , 'Oct-Mar(0)']     

In [20]:
plotting_function(precip_composite_verydry, hgt_composite_verydry, 
                  uwnd_composite_verydry, vwnd_composite_verydry, 
                  gpcp_season_members_verydry, 'GPCP', '9_panel_Very_Dry', './' ,save = 'yes', field = 'precip')

plotting_function(sst_composite_verydry, hgt_composite_verydry, 
                  uwnd_composite_verydry, vwnd_composite_verydry, 
                  gpcp_season_members_verydry, 'ERSST', '9_panel_Very_Dry', './' ,save = 'yes', field = 'sst')

/home/alejandro/anaconda3/lib/python3.5/site-packages/mpl_toolkits/basemap/__init__.py:3644: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  xx = x[x.shape[0]/2,:]
/home/alejandro/anaconda3/lib/python3.5/site-packages/mpl_toolkits/basemap/__init__.py:3542: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  xx = x[x.shape[0]/2,:]


In [21]:
plotting_function(precip_composite_nondry, hgt_composite_nondry, 
                  uwnd_composite_nondry, vwnd_composite_nondry, 
                  gpcp_season_members_nondry, 'GPCP', '9_panel_Non_Dry', './' ,save = 'yes', field = 'precip')

plotting_function(sst_composite_nondry, hgt_composite_nondry, 
                  uwnd_composite_nondry, vwnd_composite_nondry, 
                  gpcp_season_members_nondry, 'ERSST', '9_panel_Non_Dry', './' ,save = 'yes', field = 'sst')

/home/alejandro/anaconda3/lib/python3.5/site-packages/mpl_toolkits/basemap/__init__.py:3644: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  xx = x[x.shape[0]/2,:]
/home/alejandro/anaconda3/lib/python3.5/site-packages/mpl_toolkits/basemap/__init__.py:3542: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  xx = x[x.shape[0]/2,:]


In [22]:
plotting_function(precip_composite_lanina, hgt_composite_lanina, 
                  uwnd_composite_lanina, vwnd_composite_lanina, 
                  gpcp_season_members_lanina, 'GPCP', '9_panel_lanina', './' ,save = 'yes', field = 'precip')

plotting_function(sst_composite_lanina, hgt_composite_lanina, 
                  uwnd_composite_lanina, vwnd_composite_lanina, 
                  gpcp_season_members_lanina, 'ERSST', '9_panel_lanina', './' ,save = 'yes', field = 'sst')

/home/alejandro/anaconda3/lib/python3.5/site-packages/mpl_toolkits/basemap/__init__.py:3644: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  xx = x[x.shape[0]/2,:]
/home/alejandro/anaconda3/lib/python3.5/site-packages/mpl_toolkits/basemap/__init__.py:3542: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  xx = x[x.shape[0]/2,:]


# Plot the maps for individual composite members

In [ ]:
# precip_composite_verydry = np.nanmean(precip_composite_members_verydry, axis=0)
# sst_composite_verydry = np.nanmean(sst_composite_members_verydry, axis=0)
# uwnd_composite_verydry = np.nanmean(uwnd_composite_members_verydry, axis=0)
# vwnd_composite_verydry = np.nanmean(vwnd_composite_members_verydry, axis=0)
# hgt_composite_verydry = np.nanmean(hgt_composite_members_verydry, axis=0)

In [23]:
target_dir = "./composite_member_maps/"

for year in year_list:


        sst_colorfield= sst_composite_members_verydry[year_range.index(year)]

        precip_color_field= precip_composite_members_verydry[year_range.index(year)]
        
        uwnd= uwnd_composite_members_verydry[year_range.index(year)]
        vwnd = vwnd_composite_members_verydry[year_range.index(year)]
        hgt_field = hgt_composite_members_verydry[year_range.index(year)]

        plotting_function(precip_color_field, hgt_field, 
                          uwnd, vwnd, 
                          gpcp_season_members_verydry, 
                         'GPCP', str(year), target_dir ,save = 'yes', field = 'precip')

        plotting_function(sst_colorfield, hgt_field, 
                          uwnd, vwnd, 
                          gpcp_season_members_verydry, 
                          'ERSST', str(year), target_dir ,save = 'yes', field = 'sst')

/home/alejandro/anaconda3/lib/python3.5/site-packages/mpl_toolkits/basemap/__init__.py:3644: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  xx = x[x.shape[0]/2,:]
/home/alejandro/anaconda3/lib/python3.5/site-packages/mpl_toolkits/basemap/__init__.py:3542: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  xx = x[x.shape[0]/2,:]


In [24]:
target_dir = "./non_dry_composite_member_maps/"

for year in non_dry_year_list:


        sst_colorfield= sst_composite_members_nondry[year_range.index(year)]

        precip_color_field= precip_composite_members_nondry[year_range.index(year)]
        
        uwnd= uwnd_composite_members_nondry[year_range.index(year)]
        vwnd = vwnd_composite_members_nondry[year_range.index(year)]
        hgt_field = hgt_composite_members_nondry[year_range.index(year)]

        plotting_function(precip_color_field, hgt_field, 
                          uwnd, vwnd, 
                          gpcp_season_members_verydry, 
                         'GPCP', str(year), target_dir ,save = 'yes', field = 'precip')

        plotting_function(sst_colorfield, hgt_field, 
                          uwnd, vwnd, 
                          gpcp_season_members_verydry, 
                          'ERSST', str(year), target_dir ,save = 'yes', field = 'sst')

/home/alejandro/anaconda3/lib/python3.5/site-packages/mpl_toolkits/basemap/__init__.py:3644: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  xx = x[x.shape[0]/2,:]
/home/alejandro/anaconda3/lib/python3.5/site-packages/mpl_toolkits/basemap/__init__.py:3542: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  xx = x[x.shape[0]/2,:]


In [25]:
target_dir = "./lanina_composite_member_maps/"

for year in la_nina_list:


        sst_colorfield= sst_composite_members_lanina[year_range.index(year)]

        precip_color_field= precip_composite_members_lanina[year_range.index(year)]
        
        uwnd= uwnd_composite_members_lanina[year_range.index(year)]
        vwnd = vwnd_composite_members_lanina[year_range.index(year)]
        hgt_field = hgt_composite_members_lanina[year_range.index(year)]

        plotting_function(precip_color_field, hgt_field, 
                          uwnd, vwnd, 
                          gpcp_season_members_verydry, 
                         'GPCP', str(year), target_dir ,save = 'yes', field = 'precip')

        plotting_function(sst_colorfield, hgt_field, 
                          uwnd, vwnd, 
                          gpcp_season_members_verydry, 
                          'ERSST', str(year), target_dir ,save = 'yes', field = 'sst')

/home/alejandro/anaconda3/lib/python3.5/site-packages/mpl_toolkits/basemap/__init__.py:3644: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  xx = x[x.shape[0]/2,:]
/home/alejandro/anaconda3/lib/python3.5/site-packages/mpl_toolkits/basemap/__init__.py:3542: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  xx = x[x.shape[0]/2,:]


# Make dry season comparison 6 panel figure

In [26]:
#here basemap is producing a strange behaviour
m = Basemap(projection='merc', llcrnrlat=-50.1, urcrnrlat=50.01,
                    llcrnrlon=80, urcrnrlon=300, lat_ts=0, resolution='c')

with open('mapcache.pk', mode='wb') as f:
    pickle.dump(m, f)

## Tweak the subplot specifications.  

subplotparams = dict(left=0.03, right=0.88,
                     bottom=0.1, top=0.8,
                     wspace=0.05, hspace=0.001)


fig, axs = plt.subplots(nrows = 2, ncols =3, #sharex=True,
                    figsize=(13, 7.8),
                    gridspec_kw=subplotparams,
                   )


with open('mapcache.pk', 'rb') as f:
    m = pickle.load(f)
    
parallels = np.arange(-90, 90, 30)
meridians = np.arange(-180, 180, 60)
  
cmap = cm.coolwarm
bounds = [-2, -1.5,-1.25, -1, -0.75, -0.5, -0.25, 0, 0.25, 0.5, 0.75, 1, 1.25, 1.5, 2]

m.ax = axs.flat[0]

x, y = m(*np.meshgrid(ersst_longitudes, ersst_latitudes))
im = m.contourf(x, y, np.squeeze(sst_composite_verydry[13,:,:]), levels=bounds, cmap=cmap, extend='both')

x_hgt, y_hgt = m(*np.meshgrid(ncep_longitudes, ncep_latitudes))
#hgt_bounds = np.arange(-40,40,1)
hgt_bounds = [-80, -70,-60, -50, -40, 
              -30, -20, -10,-8, -6, 
              -4, -2, -1,0, 1, 2, 4, 
              6, 8, 10, 20, 30,
              40, 50, 60, 70, 80]
hgt_bound_0 = [0]
im_hgt = m.contour(x_hgt, y_hgt, hgt_composite_verydry[13,:,:], levels=hgt_bounds,
                   linewidths=0.5, colors='k')
im_hgt = m.contour(x_hgt, y_hgt, hgt_composite_verydry[13,:,:], levels=hgt_bound_0,
               linewidths=0.9, colors='k')

# transform vectors to projection grid.
uproj, vproj, xx, yy = m.rotate_vector(uwnd_composite_verydry[13,:,:], 
                                       vwnd_composite_verydry[13,:,:], 
                                       ncep_longitudes, 
                                       ncep_latitudes,
                                       returnxy=True)
# now plot every other vector
Q = m.quiver(xx[::2,::2], yy[::2,::2], 
             uproj[::2,::2], vproj[::2,::2],
             scale=20, scale_units='inches')

m.drawcoastlines()
m.drawparallels(parallels, labels = [1, 0, 0, 1], fontsize=8)
m.drawmeridians(meridians, labels = [1, 0, 0, 1], fontsize=8)
m.ax.set_title('Dry Composite')

m.ax = axs.flat[1]

x, y = m(*np.meshgrid(ersst_longitudes, ersst_latitudes))
im = m.contourf(x, y, np.squeeze(sst_composite_lanina[13,:,:]), levels=bounds, cmap=cmap, extend='both')

x_hgt, y_hgt = m(*np.meshgrid(ncep_longitudes, ncep_latitudes))
#hgt_bounds = np.arange(-40,40,1)
hgt_bounds = [-80, -70,-60, -50, -40, 
              -30, -20, -10,-8, -6, 
              -4, -2, -1,0, 1, 2, 4, 
              6, 8, 10, 20, 30,
              40, 50, 60, 70, 80]
hgt_bound_0 = [0]
im_hgt = m.contour(x_hgt, y_hgt, hgt_composite_lanina[13,:,:], levels=hgt_bounds,
                   linewidths=0.5, colors='k')
im_hgt = m.contour(x_hgt, y_hgt, hgt_composite_lanina[13,:,:], levels=hgt_bound_0,
               linewidths=0.9, colors='k')

# transform vectors to projection grid.
uproj, vproj, xx, yy = m.rotate_vector(uwnd_composite_lanina[13,:,:], 
                                       vwnd_composite_lanina[13,:,:], 
                                       ncep_longitudes, 
                                       ncep_latitudes,
                                       returnxy=True)
# now plot every other vector
Q = m.quiver(xx[::2,::2], yy[::2,::2], 
             uproj[::2,::2], vproj[::2,::2],
             scale=20, scale_units='inches')

m.drawcoastlines()
m.drawparallels(parallels, labels = [0, 0, 0, 0], fontsize=8)
m.drawmeridians(meridians, labels = [1, 0, 0, 1], fontsize=8)
m.ax.set_title('La Nina Composite')

m.ax = axs.flat[2]

x, y = m(*np.meshgrid(ersst_longitudes, ersst_latitudes))
im = m.contourf(x, y, np.squeeze(sst_composite_nondry[13,:,:]), levels=bounds, cmap=cmap, extend='both')

x_hgt, y_hgt = m(*np.meshgrid(ncep_longitudes, ncep_latitudes))
#hgt_bounds = np.arange(-40,40,1)
hgt_bounds = [-80, -70,-60, -50, -40, 
              -30, -20, -10,-8, -6, 
              -4, -2, -1,0, 1, 2, 4, 
              6, 8, 10, 20, 30,
              40, 50, 60, 70, 80]
hgt_bound_0 = [0]
im_hgt = m.contour(x_hgt, y_hgt, hgt_composite_nondry[13,:,:], levels=hgt_bounds,
                   linewidths=0.5, colors='k')
im_hgt = m.contour(x_hgt, y_hgt, hgt_composite_nondry[13,:,:], levels=hgt_bound_0,
               linewidths=0.9, colors='k')

# transform vectors to projection grid.
uproj, vproj, xx, yy = m.rotate_vector(uwnd_composite_nondry[13,:,:], 
                                       vwnd_composite_nondry[13,:,:], 
                                       ncep_longitudes, 
                                       ncep_latitudes,
                                       returnxy=True)
# now plot every other vector
Q = m.quiver(xx[::2,::2], yy[::2,::2], 
             uproj[::2,::2], vproj[::2,::2],
             scale=20, scale_units='inches')

m.drawcoastlines()
m.drawparallels(parallels, labels = [0, 0, 0, 0], fontsize=8)
m.drawmeridians(meridians, labels = [1, 0, 0, 1], fontsize=8)
m.ax.set_title('Non-Dry Composite')

#  This method of making an Axes for the cbar interacts very badly with
#  basemap, so just make one manually.  This also gives more control, so
#  it looks nicer.
left = subplotparams['right'] + 0.02
bottom = subplotparams['bottom'] + 0.1 + (subplotparams['top'] - subplotparams['bottom'] - 0.1)/2
width = 0.015
height = (subplotparams['top'] - subplotparams['bottom'] - 0.16) /2

cax = fig.add_axes([left, bottom, width, height])
cb = plt.colorbar(im, cax=cax)
cb.set_label('SST Anomaly C')
#fig.delaxes(cax)

## EF: The quiverkey needs to be made using an axes in which a quiver is
##     drawn so that it has the right transform information.  Otherwise
##     it won't get the length right.
qkx = left + (1 - left) / 6
qky = bottom + height + 0.025
qk = axs.flat[2].quiverkey(Q, qkx, qky, 10, '10 m/s', 
                   coordinates='figure',
                   labelpos='N',
                   labelsep=0.07, 
                   fontproperties=dict(size='small'),
                  )




cmap = plt.get_cmap('BrBG')
bounds = [-2.5,-2, -1.5,-1.25, -1, -0.75, -0.5, -0.25, 0, 0.25, 0.5, 0.75, 1, 1.25, 1.5, 2, 2.5]

m.ax = axs.flat[3]

x, y = m(*np.meshgrid(PREC_precip_longitudes, PREC_precip_latitudes))
im = m.contourf(x, y, precip_composite_verydry[13,:,:], levels=bounds, cmap=cmap,
                    extend='both')

x_hgt, y_hgt = m(*np.meshgrid(ncep_longitudes, ncep_latitudes))
#hgt_bounds = np.arange(-40,40,1)
hgt_bounds = [-80, -70,-60, -50, -40, 
              -30, -20, -10,-8, -6, 
              -4, -2, -1,0, 1, 2, 4, 
              6, 8, 10, 20, 30,
              40, 50, 60, 70, 80]
hgt_bound_0 = [0]
im_hgt = m.contour(x_hgt, y_hgt, hgt_composite_verydry[13,:,:], levels=hgt_bounds,
                   linewidths=0.5, colors='k')
im_hgt = m.contour(x_hgt, y_hgt, hgt_composite_verydry[13,:,:], levels=hgt_bound_0,
               linewidths=0.9, colors='k')

# transform vectors to projection grid.
uproj, vproj, xx, yy = m.rotate_vector(uwnd_composite_verydry[13,:,:], 
                                       vwnd_composite_verydry[13,:,:], 
                                       ncep_longitudes, 
                                       ncep_latitudes,
                                       returnxy=True)
# now plot every other vector
Q = m.quiver(xx[::2,::2], yy[::2,::2], 
             uproj[::2,::2], vproj[::2,::2],
             scale=20, scale_units='inches')

m.drawcoastlines()
m.drawparallels(parallels, labels = [1, 0, 0, 1], fontsize=8)
m.drawmeridians(meridians, labels = [1, 0, 0, 1], fontsize=8)

m.ax = axs.flat[4]

x, y = m(*np.meshgrid(PREC_precip_longitudes, PREC_precip_latitudes))
im = m.contourf(x, y, precip_composite_lanina[13,:,:], levels=bounds, cmap=cmap,
                    extend='both')

x_hgt, y_hgt = m(*np.meshgrid(ncep_longitudes, ncep_latitudes))
#hgt_bounds = np.arange(-40,40,1)
hgt_bounds = [-80, -70,-60, -50, -40, 
              -30, -20, -10,-8, -6, 
              -4, -2, -1,0, 1, 2, 4, 
              6, 8, 10, 20, 30,
              40, 50, 60, 70, 80]
hgt_bound_0 = [0]
im_hgt = m.contour(x_hgt, y_hgt, hgt_composite_lanina[13,:,:], levels=hgt_bounds,
                   linewidths=0.5, colors='k')
im_hgt = m.contour(x_hgt, y_hgt, hgt_composite_lanina[13,:,:], levels=hgt_bound_0,
               linewidths=0.9, colors='k')

# transform vectors to projection grid.
uproj, vproj, xx, yy = m.rotate_vector(uwnd_composite_lanina[13,:,:], 
                                       vwnd_composite_lanina[13,:,:], 
                                       ncep_longitudes, 
                                       ncep_latitudes,
                                       returnxy=True)
# now plot every other vector
Q = m.quiver(xx[::2,::2], yy[::2,::2], 
             uproj[::2,::2], vproj[::2,::2],
             scale=20, scale_units='inches')

m.drawcoastlines()
m.drawparallels(parallels, labels = [0, 0, 0, 0], fontsize=8)
m.drawmeridians(meridians, labels = [1, 0, 0, 1], fontsize=8)

m.ax = axs.flat[5]

x, y = m(*np.meshgrid(PREC_precip_longitudes, PREC_precip_latitudes))
im = m.contourf(x, y, precip_composite_nondry[13,:,:], levels=bounds, cmap=cmap,
                    extend='both')

x_hgt, y_hgt = m(*np.meshgrid(ncep_longitudes, ncep_latitudes))
#hgt_bounds = np.arange(-40,40,1)
hgt_bounds = [-80, -70,-60, -50, -40, 
              -30, -20, -10,-8, -6, 
              -4, -2, -1,0, 1, 2, 4, 
              6, 8, 10, 20, 30,
              40, 50, 60, 70, 80]
hgt_bound_0 = [0]
im_hgt = m.contour(x_hgt, y_hgt, hgt_composite_nondry[13,:,:], levels=hgt_bounds,
                   linewidths=0.5, colors='k')
im_hgt = m.contour(x_hgt, y_hgt, hgt_composite_nondry[13,:,:], levels=hgt_bound_0,
               linewidths=0.9, colors='k')

# transform vectors to projection grid.
uproj, vproj, xx, yy = m.rotate_vector(uwnd_composite_nondry[13,:,:], 
                                       vwnd_composite_nondry[13,:,:], 
                                       ncep_longitudes, 
                                       ncep_latitudes,
                                       returnxy=True)
# now plot every other vector
Q = m.quiver(xx[::2,::2], yy[::2,::2], 
             uproj[::2,::2], vproj[::2,::2],
             scale=20, scale_units='inches')

m.drawcoastlines()
m.drawparallels(parallels, labels = [0, 0, 0, 0], fontsize=8)
m.drawmeridians(meridians, labels = [1, 0, 0, 1], fontsize=8)

#  This method of making an Axes for the cbar interacts very badly with
#  basemap, so just make one manually.  This also gives more control, so
#  it looks nicer.
left = subplotparams['right'] + 0.02
bottom = subplotparams['bottom'] + 0.04
width = 0.015
height = (subplotparams['top'] - subplotparams['bottom'] - 0.15) /2

cax = fig.add_axes([left, bottom, width, height])
cb = plt.colorbar(im, cax=cax)
cb.set_label('Precip Anomaly mm/day')


## EF: The quiverkey needs to be made using an axes in which a quiver is
##     drawn so that it has the right transform information.  Otherwise
##     it won't get the length right.
qkx = left + (1 - left) / 6
qky = bottom + height + 0.025
qk = axs.flat[5].quiverkey(Q, qkx, qky, 10, '10 m/s', 
                   coordinates='figure',
                   labelpos='N',
                   labelsep=0.07, 
                   fontproperties=dict(size='small'),
                  )

plt.suptitle('Dec to May Anomalies')

fig.savefig("compsrison_maps.pdf")
fig.savefig("compsrison_maps.png")
plt.close()# no need for dpi

/home/alejandro/anaconda3/lib/python3.5/site-packages/mpl_toolkits/basemap/__init__.py:3644: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  xx = x[x.shape[0]/2,:]
/home/alejandro/anaconda3/lib/python3.5/site-packages/mpl_toolkits/basemap/__init__.py:3542: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  xx = x[x.shape[0]/2,:]


# ===============================================================
# STOP
# ===============================================================

In [27]:
print(np.shape(sst_composite_verydry[13,:,:]))

(89, 180)


In [24]:
colorfield  = np.empty([6,np.shape(sst_composite_verydry)[1],np.shape(sst_composite_verydry)[2]])
uwndfield  = np.empty([6,np.shape(uwnd_composite_verydry)[1],np.shape(uwnd_composite_verydry)[2]])
vwndfield  = np.empty([6,np.shape(vwnd_composite_verydry)[1],np.shape(vwnd_composite_verydry)[2]])
hgtfield  = np.empty([6,np.shape(hgt_composite_verydry)[1],np.shape(hgt_composite_verydry)[2]])

colorfield[0,:,:] = sst_composite_verydry[13,:,:]
colorfield[1,:,:] = sst_composite_lanina[13,:,:]
colorfield[2,:,:] = sst_composite_non_dry[13,:,:]
colorfield[3,:,:] = precip_composite_verydry[13,:,:]
colorfield[4,:,:] = sst_composite_lanina[13,:,:]
colorfield[5,:,:] = sst_composite_non_dry[13,:,:]


In [9]:
box_coords =[]

year_list = [1978, 1984, 1988, 1996, 1999, 2000, 2001, 2008, 2010, 2013]

season_list = ['DJF' , 'JFM' , 'FMA',
               'MAM' , 'AMJ' , 'MJJ',
               'JJA' , 'JAS' , 'ASO',
               'SON' , 'OND' , 'NDJ']

target_dir = "./composite_member_maps/"

In [10]:
#print(sst_seasonal_anom.s_anom.shape)
#print(sst_seasonal_anom2.shape)

In [11]:
#np.ma.allclose(sst_seasonal_anom.s_anom, sst_seasonal_anom2, masked_equal = True)

In [12]:
#print(precip_seasonal_anom.s_anom[-1])

#print('--------------------------------')

#print(uwnd_seasonal_anom.s_anom[-1])

In [ ]:
def comp_plotting_function(color_field, contour_field, u_wnd, v_wnd, 
                           dry_member_counter, wet_member_counter, year_list, 
                           color_field_name, case_name, target_dir, box_coords =[], save = 'yes', field = 'sst'):    
    
    #here basemap is producing a strange behaviour
    m = Basemap(projection='merc', llcrnrlat=-50.1, urcrnrlat=50.01,
                        llcrnrlon=80, urcrnrlon=300, lat_ts=0, resolution='c')

    with open('mapcache.pk', mode='wb') as f:
        pickle.dump(m, f)

    ## Tweak the subplot specifications.  

    subplotparams = dict(left=0.03, right=0.88,
                         bottom=0.015, top=0.94,
                         wspace=0.05, hspace=0.001)


    fig, axs = plt.subplots(3,3, #sharex=True,
                        figsize=(13, 7.8),
                        gridspec_kw=subplotparams,
                       )

    
    mm = 1

    for year, pax in zip(year_list, axs.flat):

        with open('mapcache.pk', 'rb') as f:
            m = pickle.load(f)
        m.ax = pax
        
        
        sst_selection = ((sst_ca.ymdhms[:, 0] == year))
        sst_selection = np.nonzero(sst_selection)[0]
        sst_colorfield= sst_seasonal_anom.s_anom[sst_selection]

        PREC_precip_selection = ((PREC_precip_ca.ymdhms[:, 0] == year))
        PREC_precip_selection = np.nonzero(PREC_precip_selection)[0]
        PREC_precip_color_field = PREC_precip_seasonal_anom.s_anom[PREC_precip_selection]

        selection = ((uwnd_ca.ymdhms[:, 0] == year))
        selection = np.nonzero(selection)[0]
        uwnd= uwnd_seasonal_anom.s_anom[selection]
        vwnd = vwnd_seasonal_anom.s_anom[selection]
        hgt_field = hgt_seasonal_anom.s_anom[selection]
        
        if field == 'precip':
            #reverse the coolwarm palatte to make blue rainy and red dry
            cmap = plt.get_cmap('BrBG')
            bounds = [-2.5,-2, -1.5,-1.25, -1, -0.75, -0.5, -0.25, 0, 0.25, 0.5, 0.75, 1, 1.25, 1.5, 2, 2.5]


            x, y = m(*np.meshgrid(PREC_precip_longitudes, PREC_precip_latitudes))
            im = m.contourf(x, y, color_field[mon,:,:], levels=bounds, cmap=cmap,
                            extend='both')

        elif field == 'sst':
                #reverse the coolwarm palatte to make blue rainy and red dry
                cmap = cm.coolwarm
                bounds = [-2, -1.5,-1.25, -1, -0.75, -0.5, -0.25, 0, 0.25, 0.5, 0.75, 1, 1.25, 1.5, 2]

                x, y = m(*np.meshgrid(ersst_longitudes, ersst_latitudes))
                im = m.contourf(x, y, color_field[mon,:,:], levels=bounds, cmap=cmap,
                                extend='both')
        #------------------------------------------------

        x_hgt, y_hgt = m(*np.meshgrid(ncep_longitudes, ncep_latitudes))
        #hgt_bounds = np.arange(-40,40,1)
        hgt_bounds = [-80, -70,-60, -50, -40, 
                      -30, -20, -10,-8, -6, 
                      -4, -2, -1,0, 1, 2, 4, 
                      6, 8, 10, 20, 30,
                      40, 50, 60, 70, 80]
        hgt_bound_0 = [0]
        im_hgt = m.contour(x_hgt, y_hgt, contour_field[mon,:,:], levels=hgt_bounds,
                           linewidths=0.5, colors='k')
        im_hgt = m.contour(x_hgt, y_hgt, contour_field[mon,:,:], levels=hgt_bound_0,
                       linewidths=0.9, colors='k')

        # transform vectors to projection grid.
        uproj, vproj, xx, yy = m.rotate_vector(u_wnd[mon,:,:], 
                                               v_wnd[mon,:,:], 
                                               ncep_longitudes, 
                                               ncep_latitudes,
                                               returnxy=True)
        # now plot every other vector
        Q = m.quiver(xx[::2,::2], yy[::2,::2], 
                     uproj[::2,::2], vproj[::2,::2],
                     scale=20, scale_units='inches')

        m.drawcoastlines()
        
        if box_coords in argv:
            x1,y1 = m(box_coords[2], box_coords[1])
            x2,y2 = m(box_coords[2], box_coords[0])
            x3,y3 = m(box_coords[3], box_coords[0])
            x4,y4 = m(box_coords[3], box_coords[1])
            
            p = Polygon([(x1,y1), (x2,y2), (x3,y3), (x4,y4)], 
                        edgecolor='b', facecolor = 'none', linewidth = 2)
            m.ax.add_patch(p)

        
        parallels = np.arange(-90, 90, 30)
        meridians = np.arange(-180, 180, 60)
        if pax in axs.flat[::3]:
            m.drawparallels(parallels, labels = [1, 0, 0, 1], fontsize=8)
        else:
            m.drawparallels(parallels, labels = [0, 0, 0, 0], fontsize=8)

        if pax in axs.flat[:6]:
            m.drawmeridians(meridians, labels = [0, 0, 0, 0], fontsize=8)
        if pax in axs.flat[6:]:
            m.drawmeridians(meridians, labels = [1, 0, 0, 1], fontsize=8)

        #m.drawparallels(parallels, labels = [1, 0, 0, 1], fontsize=8)
        #m.drawmeridians(meridians, labels = [1, 0, 0, 1], fontsize=8)

        #pax.set_title(color_field_name+" Seasonal Anom " +season_list[mon])
        pax.set_title(season_list[mon])

    #cax, kw = mpl.colorbar.make_axes([ax for ax in axs.flat])
    ##  This method of making an Axes for the cbar interacts very badly with
    ##  basemap, so just make one manually.  This also gives more control, so
    ##  it looks nicer.
    left = subplotparams['right'] + 0.02
    bottom = subplotparams['bottom'] + 0.05
    width = 0.015
    height = subplotparams['top'] - subplotparams['bottom'] - 0.1

    cax = fig.add_axes([left, bottom, width, height])
    cb = plt.colorbar(im, cax=cax)
    
    if field == 'precip':
        cb.set_label('Precip Anomaly mm/day')
    elif field == 'sst':
        cb.set_label('SST Anomaly C')
    

    ## EF: The quiverkey needs to be made using an axes in which a quiver is
    ##     drawn so that it has the right transform information.  Otherwise
    ##     it won't get the length right.
    qkx = left + (1 - left) / 4
    qky = bottom + height + 0.025
    qk = pax.quiverkey(Q, qkx, qky, 10, '10 m/s', 
                       coordinates='figure',
                       labelpos='N',
                       labelsep=0.07, 
                       fontproperties=dict(size='small'),
                      )
    
    if field == 'precip':
        plt.suptitle(case_name+' '+color_field_name+' Precip 850HGT WND seasonal anomaly')
    elif field == 'sst':
        plt.suptitle(case_name+' '+color_field_name+' Precip 850HGT WND seasonal anomaly')
    
    
    if save == 'yes':

        fig.savefig(target_dir+str(case_name)+"_"+color_field_name+"_PRECIP_850HGT_WND_seasonal_anomaly.pdf")
        fig.savefig(target_dir+str(case_name)+"_"+color_field_name+"_PRECIP_850HGT_WND_seasonal_anomaly.png")
        #fig.savefig("VERY_DRY_COMPOSITE_SST_850HGT_WND_seasonal_anomaly_%04d-%02d.pdf" % (year, mon))
        plt.close()# no need for dpi

# Rainfall anomaly pattern correlation calculation

In [13]:
lat_sl = rangeslice(precip_latitudes, (-5, 15.001))

#lon goes from 0 to 360
lon_sl = rangeslice(precip_longitudes, (120, 240.001))

start_year = 1979
end_year = 2015

#Calculate the total number of pattern correlations possible
total_year_number = end_year - start_year +1

total_dims = comb(total_year_number,2, exact = True)


c_precip = np.empty((total_dims,12))

x=0
for y in range(start_year, end_year):
    #print(y)
    for i in range(1,end_year - y+1):
        #print(y+i)
        for m in range(12):
            #
            #t1 = np.nonzero(dat.dday_1800 == to_day(1800, y, m+1))[0][0]
            t1 = ((precip_ca.ymdhms[:, 0] == y) & (precip_ca.ymdhms[:, 1] == m+1))
            #t2 = np.nonzero(dat.dday_1800 == to_day(1800, y+i, m+1))[0][0]
            t2 = ((precip_ca.ymdhms[:, 0] == y+i) & (precip_ca.ymdhms[:, 1] == m+1))
                
            #
            uav_1 = precip_seasonal_anom.s_anom[t1, lat_sl, lon_sl]
            uav_2 = precip_seasonal_anom.s_anom[t2, lat_sl, lon_sl]
            
            y1 = uav_1.compressed()
            y2 = uav_2.compressed()
            try:
                c_precip[x,m] = np.corrcoef(y1, y2)[0, 1]
            except ValueError:
                c_precip[x,m] = np.nan
            
            if c_precip[x,m] > 1:
                print(y)
                print(y+i)
        
        x=x+1

#-----------------------------------------------------------------------------------------------------------------
PREC_start_year = 1966
PREC_end_year = 2015

#Calculate the total number of pattern correlations possible
PREC_total_year_number = PREC_end_year - PREC_start_year +1

PREC_total_dims = comb(PREC_total_year_number,2, exact = True)


c_PREC_precip = np.empty((PREC_total_dims,12))

x=0
for y in range(PREC_start_year, PREC_end_year):
    #print(y)
    for i in range(1,PREC_end_year - y+1):
        #print(y+i)
        for m in range(12):
            #
            #t1 = np.nonzero(dat.dday_1800 == to_day(1800, y, m+1))[0][0]
            t1 = ((PREC_precip_ca.ymdhms[:, 0] == y) & (PREC_precip_ca.ymdhms[:, 1] == m+1))
            #t2 = np.nonzero(dat.dday_1800 == to_day(1800, y+i, m+1))[0][0]
            t2 = ((PREC_precip_ca.ymdhms[:, 0] == y+i) & (PREC_precip_ca.ymdhms[:, 1] == m+1))
                
            #
            uav_1 = PREC_precip_seasonal_anom.s_anom[t1, lat_sl, lon_sl]
            uav_2 = PREC_precip_seasonal_anom.s_anom[t2, lat_sl, lon_sl]
            
            y1 = uav_1.compressed()
            y2 = uav_2.compressed()
            try:
                c_PREC_precip[x,m] = np.corrcoef(y1, y2)[0, 1]
            except ValueError:
                c_PREC_precip[x,m] = np.nan
            
            if c_PREC_precip[x,m] > 1:
                print(y)
                print(y+i)
        
        x=x+1

/home/alejandro/anaconda3/lib/python3.5/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)
/home/alejandro/anaconda3/lib/python3.5/site-packages/numpy/lib/function_base.py:2079: RuntimeWarning: Degrees of freedom <= 0 for slice
  warnings.warn("Degrees of freedom <= 0 for slice", RuntimeWarning)


In [14]:
pc90 = np.nanpercentile(c_precip, 90, axis=0)
pc95 = np.nanpercentile(c_precip, 95, axis=0)
np.set_printoptions(precision=2, suppress=True)
print(pc90[:])
print(pc95[:])

[ 0.68  0.65  0.63  0.58  0.52  0.5   0.47  0.49  0.54  0.58  0.64  0.67]
[ 0.77  0.75  0.73  0.66  0.61  0.58  0.57  0.61  0.64  0.67  0.73  0.76]


In [15]:
PREC_pc90 = np.nanpercentile(c_PREC_precip, 90, axis=0)
PREC_pc95 = np.nanpercentile(c_PREC_precip, 95, axis=0)
np.set_printoptions(precision=2, suppress=True)
print(PREC_pc90[:])
print(PREC_pc95[:])

[ 0.85  0.83  0.8   0.79  0.73  0.66  0.65  0.63  0.69  0.77  0.81  0.83]
[ 0.91  0.92  0.9   0.87  0.81  0.77  0.73  0.75  0.78  0.84  0.88  0.91]


In [16]:
# Calculate the pattern correlation of each year to 2013
total_year_number = end_year - start_year +1

c_precip_2013 = np.empty((total_year_number,12))

x=0
for y in range(start_year, end_year+1):
    #print(y+i)
    for m in range(12):
        #
        #t1 = np.nonzero(dat.dday_1800 == to_day(1800, y, m+1))[0][0]
        t1 = ((precip_ca.ymdhms[:, 0] == y) & (precip_ca.ymdhms[:, 1] == m+1))
        #t2 = np.nonzero(dat.dday_1800 == to_day(1800, y+i, m+1))[0][0]
        t2 = ((precip_ca.ymdhms[:, 0] == 2013) & (precip_ca.ymdhms[:, 1] == m+1))

        #
        uav_1 = precip_seasonal_anom.s_anom[t1, lat_sl, lon_sl]
        uav_2 = precip_seasonal_anom.s_anom[t2, lat_sl, lon_sl]


        y1 = uav_1.compressed()
        y2 = uav_2.compressed()
        try:
            c_precip_2013[x,m] = np.corrcoef(y1, y2)[0, 1]
        except ValueError:
            c_precip_2013[x,m] = np.nan

        if c_precip_2013[x,m] > 1:
            print(y)
            print(y+i)

    x=x+1
    
# Calculate the pattern correlation of each year to 2013
PREC_total_year_number = PREC_end_year - PREC_start_year +1

c_PREC_precip_2013 = np.empty((PREC_total_year_number,12))

x=0
for y in range(PREC_start_year, PREC_end_year+1):
    #print(y+i)
    for m in range(12):
        #
        #t1 = np.nonzero(dat.dday_1800 == to_day(1800, y, m+1))[0][0]
        t1 = ((PREC_precip_ca.ymdhms[:, 0] == y) & (PREC_precip_ca.ymdhms[:, 1] == m+1))
        #t2 = np.nonzero(dat.dday_1800 == to_day(1800, y+i, m+1))[0][0]
        t2 = ((PREC_precip_ca.ymdhms[:, 0] == 2013) & (PREC_precip_ca.ymdhms[:, 1] == m+1))

        #
        uav_1 = PREC_precip_seasonal_anom.s_anom[t1, lat_sl, lon_sl]
        uav_2 = PREC_precip_seasonal_anom.s_anom[t2, lat_sl, lon_sl]


        y1 = uav_1.compressed()
        y2 = uav_2.compressed()
        try:
            c_PREC_precip_2013[x,m] = np.corrcoef(y1, y2)[0, 1]
        except ValueError:
            c_PREC_precip_2013[x,m] = np.nan

        if c_PREC_precip_2013[x,m] > 1:
            print(y)
            print(y+i)

    x=x+1

In [17]:
#print(c_precip_2013)
#print(uav_1)

In [18]:
#print(c_PREC_precip_2013)

In [19]:
gpcp_pat_cor_90_test_lev = c_precip_2013 > pc90
gpcp_pat_cor_95_test_lev = c_precip_2013 > pc95

PREC_pat_cor_90_test_lev = c_PREC_precip_2013 > PREC_pc90
PREC_pat_cor_95_test_lev = c_PREC_precip_2013 > PREC_pc95


pat_cor_90_test_lev = np.concatenate((PREC_pat_cor_90_test_lev[:13,:], gpcp_pat_cor_90_test_lev), axis = 0)
pat_cor_95_test_lev = np.concatenate((PREC_pat_cor_95_test_lev[:13,:], gpcp_pat_cor_95_test_lev), axis = 0)

In [20]:
print(np.shape(pat_cor_90_test_lev))

(50, 12)


# Here we read and process the time series data from the excel files

In [21]:
peac_station_rain = pd.ExcelFile(datadir +'PEAC_station_rainfall_database.xlsx')
print(peac_station_rain.sheet_names)

ONI_file = pd.ExcelFile(datadir +'CPC_ONI.xlsx')
print(ONI_file.sheet_names)


['Guam', 'Saipan', 'Koror', 'Yap', 'Chuuk', 'Pohnpei', 'Majuro', 'Kwajalein', 'PagoPago', 'JFM', 'FMA', 'MAM', 'AMJ', 'MJJ', 'JJA', 'JAS', 'ASO', 'SON', 'OND', 'NDJ', 'DJF']
['ONI']


In [22]:
def read_USAPI_data(file_name, island_name):
    
    #Here we read the data into python from the excel file
    raw_data= pd.read_excel(file_name, sheetname = island_name, 
                            skiprows = 1, parse_cols = "A:O")
    
    #
    raw_matrix = raw_data.as_matrix(columns=None)
    print(type(raw_matrix), raw_matrix.dtype)
    
    years =  raw_matrix[:51,1]
    station_id = raw_matrix[0,0]
    rainfall = raw_matrix[:51,3:] * 0.1
    
    rainfall[rainfall == "'9999"] = np.nan
    rainfall[rainfall == "nan"] = np.nan
    # We have to convert from an object array to floating point.
    rainfall = np.ma.masked_invalid(rainfall.astype(float))
    print('rainfall: ', rainfall.dtype, rainfall[5, 5])
 
    #Initiate the ymdhms array as an array of int values filler with zeros
    #that has the length og the total number of time steps years*months
    ymdhms = np.zeros([51*12,6],dtype = np.int)

    #here we will the first column with the year values from the excel sheet. 
    #repeated each one 12 consecutive times
    ymdhms[:,0] = np.repeat(years,12)

    #Here we create a list of the month indices
    m = np.arange(1,13)
    #we tile the list so that the entire list 1..12 repeats as many times as the number of years
    mm = np.tile(m,51)

    ymdhms[:,1] = mm

    #the day column is filled with 15
    ymdhms[:,2] = 15
    
    dday = to_day(1800, ymdhms)
    mpldays = dday_to_mpl(1800, dday)
    mpldaysformated = mpl.dates.num2date(mpldays)

    out = Bunch(island_name = island_name, station_id = station_id, rainfall=rainfall,
               ymdhms = ymdhms, mpldaysformated = mpldaysformated)
    
    return out
    #return station_id, rainfall, ymdhms, mpldaysformated
    
def read_index_data(file_name, index_name):
    
    #Here we read the data into python from the excel file
    raw_data= pd.read_excel(file_name, sheetname = index_name, skiprows = 1, parse_cols = "A:M")
    
    #
    raw_matrix = raw_data.as_matrix(columns=None)
    print(type(raw_matrix), raw_matrix.dtype)
    
    years =  raw_matrix[:,0]
    index = raw_matrix[:,1:]
    

    # We have to convert from an object array to floating point.
    index = np.ma.masked_invalid(index.astype(float))
    print('index: ', index.dtype, index[5, 5])
 
    #Initiate the ymdhms array as an array of int values filler with zeros
    #that has the length og the total number of time steps years*months
    ymdhms = np.zeros([67*12,6],dtype = np.int)

    #here we will the first column with the year values from the excel sheet. 
    #repeated each one 12 consecutive times
    ymdhms[:,0] = np.repeat(years,12)

    #Here we create a list of the month indices
    m = np.arange(1,13)
    #we tile the list so that the entire list 1..12 repeats as many times as the number of years
    mm = np.tile(m,67)

    ymdhms[:,1] = mm

    #the day column is filled with 15
    ymdhms[:,2] = 15
    
    dday = to_day(1800, ymdhms)
    mpldays = dday_to_mpl(1800, dday)
    mpldaysformated = mpl.dates.num2date(mpldays)

    out = Bunch(index_name = index_name, index = index,
               ymdhms = ymdhms, mpldaysformated = mpldaysformated)
    
    return out
    #return station_id, rainfall, ymdhms, mpldaysformated

def seasonal_anomaly_old(m_anom):
    s_anom = np.ma.zeros(m_anom.shape, float)
    s_anom[1:-1] = (m_anom[:-2] + m_anom[1:-1] + m_anom[2:]) / 3
    s_anom[0] = (m_anom[0] + m_anom[1]) / 2
    s_anom[-1] = (m_anom[-2] + m_anom[-1]) / 2
    return s_anom

In [24]:
station_name_list = ["Koror", "Yap", "Guam", "Chuuk", "Pohnpei", "Kwajalein", "Majuro"]
#variable_name_list = [koror, yap, guam, chuuk, phonpei, kwajalein, majuro]

stations = Bunch()
for name in station_name_list:
    stations[name] = read_USAPI_data(peac_station_rain, name)
    
#print(stations.Kwajalein.ymdhms)

<class 'numpy.ndarray'> object
rainfall:  float64 498.1
<class 'numpy.ndarray'> object
rainfall:  float64 354.3
<class 'numpy.ndarray'> object
rainfall:  float64 128.6
<class 'numpy.ndarray'> object
rainfall:  float64 353.9
<class 'numpy.ndarray'> object
rainfall:  float64 599.9
<class 'numpy.ndarray'> object
rainfall:  float64 292.9
<class 'numpy.ndarray'> object
rainfall:  float64 340.5


In [25]:
for raindata in stations.values():
    print(raindata.island_name)
    print(np.shape(raindata.rainfall))
    raindata.monmean = raindata.rainfall.mean(axis=0)
    raindata.monstd = raindata.rainfall.std(axis=0)
    raindata.monanom = raindata.rainfall - raindata.monmean

Guam
(51, 12)
Pohnpei
(51, 12)
Yap
(51, 12)
Majuro
(51, 12)
Kwajalein
(51, 12)
Koror
(51, 12)
Chuuk
(51, 12)


In [26]:
print(np.shape(stations.Kwajalein.monanom))
kwajalein_seasonal_anom = seasonal_anomaly_old(stations.Kwajalein.monanom.ravel())

(51, 12)


In [27]:
oni = read_index_data(ONI_file, "ONI")

print(np.shape(oni.ymdhms))
print(np.shape(oni.mpldaysformated))
print(np.shape(oni.index))

<class 'numpy.ndarray'> float64
index:  float64 -0.6
(804, 6)
(804,)
(67, 12)


In [28]:
oni_selection = ((oni.ymdhms[:, 0] >= 1966))

oni_time_series = oni.index.ravel()

oni_period = oni_time_series[oni_selection]

print(type(oni.ymdhms))
print(type(oni.mpldaysformated))
print(type(oni_selection))
print(np.shape(oni_period))

mpldaysformated_period = oni.mpldaysformated[-612:]
print(len(mpldaysformated_period))

<class 'numpy.ndarray'>
<class 'list'>
<class 'numpy.ndarray'>
(612,)
612


In [29]:
fig, ax1 = plt.subplots()

ax1.plot(mpldaysformated_period, oni_period, 'r-')
ax1.set_xlabel('year')
# Make the y-axis label and tick labels match the line color.
ax1.set_ylabel('ONI', color='r')
for tl in ax1.get_yticklabels():
    tl.set_color('r')


ax2 = ax1.twinx()

ax2.plot(mpldaysformated_period, kwajalein_seasonal_anom, 'b-')
ax2.set_ylabel('Kwajalein Seasonal Rainfall Anom mm/month', color='b')
for tl in ax2.get_yticklabels():
    tl.set_color('b')
plt.show()

## stdev of the kwajalein rainfall
Now we must calculate the stdev of the seasonal anomaly. The most straight forward way to calculate this may be unraveling the monthly anomaly into a series, using the seasonal_anomaly function and reshaping the series back into a matrix, then calculate the stdev of the columns of that matrix.

In [30]:
#here we restructure the seaqsonal anomaly time series into matices for easier logical selection
kwajalein_seasonal_anom_matrix = kwajalein_seasonal_anom.reshape(51, 12)
oni_period_matrix = oni_period.reshape(51,12)

#Here we calculate the stdev of eac column of the matrix this is the stdev of seasonal rainfall anomaly for each season
kwajalein_season_std = kwajalein_seasonal_anom_matrix.std(axis=0)
#we repeat the row 51 times to have a matrix the same size as the data matrix and the oni matrix
kwajalein_season_std_matrix = np.tile(kwajalein_season_std,(51,1))


# Aplication of conditions for composite membership

## first for the rainfall and ONI conditions only

In [31]:
cond_1stdev = kwajalein_seasonal_anom_matrix <= -1* kwajalein_season_std_matrix
cond_oni = oni_period_matrix <= -0.1
cond_verydry_cool = np.logical_and(cond_1stdev, cond_oni)

cond_05stdev = kwajalein_seasonal_anom_matrix <= -0.5* kwajalein_season_std_matrix
cond_oni = oni_period_matrix <= -0.1
cond_dry_cool = np.logical_and(cond_05stdev, cond_oni)

## nos for the pattern correlation conditions too

### we use only the 90% test level

In [32]:
#cond_verydry_cool_90patcor = np.logical_and.reduce((cond_1stdev[:-1,:], cond_oni[:-1,:], pat_cor_90_test_lev))
#cond_dry_cool_90patcor = np.logical_and.reduce((cond_05stdev[:-1,:], cond_oni[:-1,:], pat_cor_90_test_lev))

cond_verydry_cool_90patcor = np.logical_and(cond_verydry_cool[:-1,:], pat_cor_90_test_lev)
cond_dry_cool_90patcor = np.logical_and(cond_dry_cool[:-1,:], pat_cor_90_test_lev)

#cond_verydry_cool_90patcor = np.logical_and(cond_1stdev[:-1,:], cond_oni[:-1,:], pat_cor_90_test_lev)
#cond_dry_cool_90patcor = np.logical_and(cond_05stdev[:-1,:], cond_oni[:-1,:], pat_cor_90_test_lev)

# Print the years selected

In [33]:
#selection_matrix = cond_dry_cool

#for i in range(len(selection_matrix.ravel())):
#    if selection_matrix.ravel()[i]:
#        print(stations.Kwajalein.ymdhms[i])

In [34]:
selection_matrix = cond_verydry_cool

for i in range(len(selection_matrix.ravel())):
    if selection_matrix.ravel()[i]:
        print(stations.Kwajalein.ymdhms[i])

[1971   11   15    0    0    0]
[1973   10   15    0    0    0]
[1973   11   15    0    0    0]
[1975    2   15    0    0    0]
[1978    7   15    0    0    0]
[1978    8   15    0    0    0]
[1978    9   15    0    0    0]
[1981    1   15    0    0    0]
[1984    4   15    0    0    0]
[1984    5   15    0    0    0]
[1984    6   15    0    0    0]
[1984    7   15    0    0    0]
[1984    8   15    0    0    0]
[1984    9   15    0    0    0]
[1988    6   15    0    0    0]
[1988    7   15    0    0    0]
[1989    7   15    0    0    0]
[1989   11   15    0    0    0]
[1996    7   15    0    0    0]
[1996    8   15    0    0    0]
[1996    9   15    0    0    0]
[1998    6   15    0    0    0]
[1999    4   15    0    0    0]
[2000    5   15    0    0    0]
[2000    6   15    0    0    0]
[2001    3   15    0    0    0]
[2001    4   15    0    0    0]
[2001    5   15    0    0    0]
[2001    6   15    0    0    0]
[2001    8   15    0    0    0]
[2008   10   15    0    0    0]
[2008   

In [87]:
selection_matrix = cond_verydry_cool_90patcor

for i in range(len(selection_matrix.ravel())):
    if selection_matrix.ravel()[i]:
        print(stations.Kwajalein.ymdhms[i])
    

[1984    6   15    0    0    0]
[1984    7   15    0    0    0]
[1984    8   15    0    0    0]
[1984    9   15    0    0    0]
[1988    6   15    0    0    0]
[1988    7   15    0    0    0]
[1996    7   15    0    0    0]
[1996    8   15    0    0    0]
[1999    4   15    0    0    0]
[2000    6   15    0    0    0]
[2001    3   15    0    0    0]
[2001    4   15    0    0    0]
[2001    5   15    0    0    0]
[2010    8   15    0    0    0]
[2013    2   15    0    0    0]
[2013    6   15    0    0    0]
[2013    7   15    0    0    0]
[2013    8   15    0    0    0]
[2013    9   15    0    0    0]
[2013   10   15    0    0    0]
[2013   11   15    0    0    0]


# Below we set up the matrices that will contain the composite members

In [38]:
season_members_verydry = np.zeros(12)
gpcp_season_members_verydry = np.zeros(12)

sst_composite_members_verydry = np.empty([50,12, len(ersst_latitudes), len(ersst_longitudes)])
sst_composite_members_verydry[:] = np.NAN

precip_composite_members_verydry = np.empty([50,12, len(precip_latitudes), len(precip_longitudes)])
precip_composite_members_verydry[:] = np.NAN

PREC_precip_composite_members_verydry = np.empty([50,12, len(PREC_precip_latitudes), len(PREC_precip_longitudes)])
PREC_precip_composite_members_verydry[:] = np.NAN

uwnd_composite_members_verydry = np.empty([50,12,len(ncep_latitudes), len(ncep_longitudes)])
uwnd_composite_members_verydry[:] = np.NAN
vwnd_composite_members_verydry = np.empty([50,12,len(ncep_latitudes), len(ncep_longitudes)])
vwnd_composite_members_verydry[:] = np.NAN
hgt_composite_members_verydry = np.empty([50,12,len(ncep_latitudes), len(ncep_longitudes)])
hgt_composite_members_verydry[:] = np.NAN

season_members_dry = np.zeros(12)
gpcp_season_members_dry = np.zeros(12)

precip_composite_members_dry = np.empty([50,12, len(precip_latitudes), len(precip_longitudes)])
precip_composite_members_dry[:] = np.NAN

sst_composite_members_dry = np.empty([50,12, len(ersst_latitudes), len(ersst_longitudes)])
sst_composite_members_dry[:] = np.NAN

PREC_precip_composite_members_dry = np.empty([51,12, len(PREC_precip_latitudes), len(PREC_precip_longitudes)])
PREC_precip_composite_members_dry[:] = np.NAN

uwnd_composite_members_dry = np.empty([50,12,len(ncep_latitudes), len(ncep_longitudes)])
uwnd_composite_members_dry[:] = np.NAN
vwnd_composite_members_dry = np.empty([50,12,len(ncep_latitudes), len(ncep_longitudes)])
vwnd_composite_members_dry[:] = np.NAN
hgt_composite_members_dry = np.empty([50,12,len(ncep_latitudes), len(ncep_longitudes)])
hgt_composite_members_dry[:] = np.NAN

In [39]:
season_members_verydry_90patcor = np.zeros(12)
gpcp_season_members_verydry_90patcor = np.zeros(12)

precip_composite_members_verydry_90patcor = np.empty([50,12, len(precip_latitudes), len(precip_longitudes)])
precip_composite_members_verydry_90patcor[:] = np.NAN

sst_composite_members_verydry_90patcor = np.empty([50,12, len(ersst_latitudes), len(ersst_longitudes)])
sst_composite_members_verydry_90patcor[:] = np.NAN

PREC_precip_composite_members_verydry_90patcor = np.empty([50,12, len(PREC_precip_latitudes), len(PREC_precip_longitudes)])
PREC_precip_composite_members_verydry_90patcor[:] = np.NAN

uwnd_composite_members_verydry_90patcor = np.empty([50,12,len(ncep_latitudes), len(ncep_longitudes)])
uwnd_composite_members_verydry_90patcor[:] = np.NAN
vwnd_composite_members_verydry_90patcor = np.empty([50,12,len(ncep_latitudes), len(ncep_longitudes)])
vwnd_composite_members_verydry_90patcor[:] = np.NAN
hgt_composite_members_verydry_90patcor = np.empty([50,12,len(ncep_latitudes), len(ncep_longitudes)])
hgt_composite_members_verydry_90patcor[:] = np.NAN

season_members_dry_90patcor = np.zeros(12)
gpcp_season_members_dry_90patcor = np.zeros(12)

precip_composite_members_dry_90patcor = np.empty([50,12, len(precip_latitudes), len(precip_longitudes)])
precip_composite_members_dry_90patcor[:] = np.NAN

sst_composite_members_dry_90patcor = np.empty([50,12, len(ersst_latitudes), len(ersst_longitudes)])
sst_composite_members_dry_90patcor[:] = np.NAN

PREC_precip_composite_members_dry_90patcor = np.empty([50,12, len(PREC_precip_latitudes), len(PREC_precip_longitudes)])
PREC_precip_composite_members_dry_90patcor[:] = np.NAN

uwnd_composite_members_dry_90patcor = np.empty([50,12,len(ncep_latitudes), len(ncep_longitudes)])
uwnd_composite_members_dry_90patcor[:] = np.NAN
vwnd_composite_members_dry_90patcor = np.empty([50,12,len(ncep_latitudes), len(ncep_longitudes)])
vwnd_composite_members_dry_90patcor[:] = np.NAN
hgt_composite_members_dry_90patcor = np.empty([50,12,len(ncep_latitudes), len(ncep_longitudes)])
hgt_composite_members_dry_90patcor[:] = np.NAN

In [40]:
#season_members_verydry_90patcor = np.zeros(12)

#sst_composite_members_verydry_90patcor = np.empty([50,12, len(ersst_latitudes), len(ersst_longitudes)])
#sst_composite_members_verydry_90patcor[:] = np.NAN
#uwnd_composite_members_verydry_90patcor = np.empty([50,12,len(ncep_latitudes), len(ncep_longitudes)])
#uwnd_composite_members_verydry_90patcor[:] = np.NAN
#vwnd_composite_members_verydry_90patcor = np.empty([50,12,len(ncep_latitudes), len(ncep_longitudes)])
#vwnd_composite_members_verydry_90patcor[:] = np.NAN
#hgt_composite_members_verydry_90patcor = np.empty([50,12,len(ncep_latitudes), len(ncep_longitudes)])
#hgt_composite_members_verydry_90patcor[:] = np.NAN

#season_members_dry_90patcor = np.zeros(12)

#sst_composite_members_dry_90patcor = np.empty([50,12, len(ersst_latitudes), len(ersst_longitudes)])
#sst_composite_members_dry_90patcor[:] = np.NAN
#uwnd_composite_members_dry_90patcor = np.empty([50,12,len(ncep_latitudes), len(ncep_longitudes)])
#uwnd_composite_members_dry_90patcor[:] = np.NAN
#vwnd_composite_members_dry_90patcor = np.empty([50,12,len(ncep_latitudes), len(ncep_longitudes)])
#vwnd_composite_members_dry_90patcor[:] = np.NAN
#hgt_composite_members_dry_90patcor = np.empty([50,12,len(ncep_latitudes), len(ncep_longitudes)])
#hgt_composite_members_dry_90patcor[:] = np.NAN

In [103]:
season_members_verydry = np.zeros(12)
gpcp_season_members_verydry = np.zeros(12)

season_members_verydry_90patcor = np.zeros(12)
gpcp_season_members_verydry_90patcor = np.zeros(12)


year_range = range(1966,2016)
short_year_range = range(1979,2016)
for year in short_year_range:
    for m in range(1,13):
        
        if cond_verydry_cool[year_range.index(year),m-1]:   
            
            
            #print(year,year_range.index(year),m,cond_verydry_cool[year_range.index(year),m-1])

            sst_selection = ((sst_ca.ymdhms[:, 0] == year) & (sst_ca.ymdhms[:, 1] == m))
            sst_selection = np.nonzero(sst_selection)[0][0]
            sst_composite_members_verydry[year_range.index(year),m-1,:,:] = sst_seasonal_anom.s_anom[sst_selection]
            
            PREC_precip_selection = ((PREC_precip_ca.ymdhms[:, 0] == year) & (PREC_precip_ca.ymdhms[:, 1] == m))
            PREC_precip_selection = np.nonzero(PREC_precip_selection)[0][0]
            PREC_precip_composite_members_verydry[year_range.index(year),m-1,:,:] = PREC_precip_seasonal_anom.s_anom[PREC_precip_selection]
            
            selection = ((uwnd_ca.ymdhms[:, 0] == year) & (uwnd_ca.ymdhms[:, 1] == m))
            selection = np.nonzero(selection)[0][0]
            uwnd_composite_members_verydry[year_range.index(year),m-1,:,:] = uwnd_seasonal_anom.s_anom[selection]
            vwnd_composite_members_verydry[year_range.index(year),m-1,:,:] = vwnd_seasonal_anom.s_anom[selection]
            hgt_composite_members_verydry[year_range.index(year),m-1,:,:] = hgt_seasonal_anom.s_anom[selection]
            
            season_members_verydry[m-1] = season_members_verydry[m-1] +1
            
            precip_selection = ((precip_ca.ymdhms[:, 0] == year) & (precip_ca.ymdhms[:, 1] == m))
            precip_selection = np.nonzero(precip_selection)[0][0]
            precip_composite_members_verydry[year_range.index(year),m-1,:,:] = precip_seasonal_anom.s_anom[precip_selection]

            gpcp_season_members_verydry[m-1] = gpcp_season_members_verydry[m-1] +1
                

#--------------------------------------------------------------------------------------------------------------------
for year in short_year_range:

    for m in range(1,13):
        
        if cond_verydry_cool_90patcor[year_range.index(year),m-1]:   
            
            #print(year,year_range.index(year),m,cond_verydry_cool[year_range.index(year),m-1])

            sst_selection = ((sst_ca.ymdhms[:, 0] == year) & (sst_ca.ymdhms[:, 1] == m))
            sst_selection = np.nonzero(sst_selection)[0][0]
            sst_composite_members_verydry_90patcor[year_range.index(year),m-1,:,:] = sst_seasonal_anom.s_anom[sst_selection]
            
            PREC_precip_selection = ((PREC_precip_ca.ymdhms[:, 0] == year) & (PREC_precip_ca.ymdhms[:, 1] == m))
            PREC_precip_selection = np.nonzero(PREC_precip_selection)[0][0]
            PREC_precip_composite_members_verydry_90patcor[year_range.index(year),m-1,:,:] = PREC_precip_seasonal_anom.s_anom[PREC_precip_selection]
            
            selection = ((uwnd_ca.ymdhms[:, 0] == year) & (uwnd_ca.ymdhms[:, 1] == m))
            selection = np.nonzero(selection)[0][0]
            uwnd_composite_members_verydry_90patcor[year_range.index(year),m-1,:,:] = uwnd_seasonal_anom.s_anom[selection]
            vwnd_composite_members_verydry_90patcor[year_range.index(year),m-1,:,:] = vwnd_seasonal_anom.s_anom[selection]
            hgt_composite_members_verydry_90patcor[year_range.index(year),m-1,:,:] = hgt_seasonal_anom.s_anom[selection]
            
            season_members_verydry_90patcor[m-1] = season_members_verydry_90patcor[m-1] +1
            
            precip_selection = ((precip_ca.ymdhms[:, 0] == year) & (precip_ca.ymdhms[:, 1] == m))
            precip_selection = np.nonzero(precip_selection)[0][0]
            precip_composite_members_verydry_90patcor[year_range.index(year),m-1,:,:] = precip_seasonal_anom.s_anom[precip_selection]

            gpcp_season_members_verydry_90patcor[m-1] = gpcp_season_members_verydry_90patcor[m-1] +1

In [104]:
#print(np.shape(PREC_precip_seasonal_anom.s_anom))
#PREC_precip_selection = ((PREC_precip_ca.ymdhms[:, 0] == 1999) & (PREC_precip_ca.ymdhms[:, 1] == m))
#PREC_precip_selection = np.nonzero(PREC_precip_selection)[0][0]
#print(PREC_precip_seasonal_anom.s_anom[PREC_precip_selection])

#year_range = range(1966,2016)
#for year in year_range:
#    print(year_range.index(year), year)

In [105]:
season_members_dry = np.zeros(12)
gpcp_season_members_dry = np.zeros(12)

season_members_dry_90patcor = np.zeros(12)
gpcp_season_members_dry_90patcor = np.zeros(12)

for year in short_year_range:
    for m in range(1,13):
        
        if cond_dry_cool[year_range.index(year),m-1]:   
            
            #print(m-1,year)
            #print(year,year_range.index(year),m,cond_verydry_cool[year_range.index(year),m-1])

            sst_selection = ((sst_ca.ymdhms[:, 0] == year) & (sst_ca.ymdhms[:, 1] == m))
            sst_selection = np.nonzero(sst_selection)[0][0]
            sst_composite_members_dry[year_range.index(year),m-1,:,:] = sst_seasonal_anom.s_anom[sst_selection]
            
            PREC_precip_selection = ((PREC_precip_ca.ymdhms[:, 0] == year) & (PREC_precip_ca.ymdhms[:, 1] == m))
            PREC_precip_selection = np.nonzero(PREC_precip_selection)[0][0]
            PREC_precip_composite_members_dry[year_range.index(year),m-1,:,:] = PREC_precip_seasonal_anom.s_anom[PREC_precip_selection]
            
            selection = ((uwnd_ca.ymdhms[:, 0] == year) & (uwnd_ca.ymdhms[:, 1] == m))
            selection = np.nonzero(selection)[0][0]
            uwnd_composite_members_dry[year_range.index(year),m-1,:,:] = uwnd_seasonal_anom.s_anom[selection]
            vwnd_composite_members_dry[year_range.index(year),m-1,:,:] = vwnd_seasonal_anom.s_anom[selection]
            hgt_composite_members_dry[year_range.index(year),m-1,:,:] = hgt_seasonal_anom.s_anom[selection]
            
            season_members_dry[m-1] = season_members_dry[m-1] +1
            

            precip_selection = ((precip_ca.ymdhms[:, 0] == year) & (precip_ca.ymdhms[:, 1] == m))
            precip_selection = np.nonzero(precip_selection)[0][0]
            precip_composite_members_dry[year_range.index(year),m-1,:,:] = precip_seasonal_anom.s_anom[precip_selection]

            gpcp_season_members_dry[m-1] = gpcp_season_members_dry[m-1] +1
                

#--------------------------------------------------------------------------------------------------------------------
for year in short_year_range:

    for m in range(1,13):
        
        if cond_dry_cool_90patcor[year_range.index(year),m-1]:   
            
            #print(year,year_range.index(year),m,cond_verydry_cool[year_range.index(year),m-1])

            sst_selection = ((sst_ca.ymdhms[:, 0] == year) & (sst_ca.ymdhms[:, 1] == m))
            sst_selection = np.nonzero(sst_selection)[0][0]
            sst_composite_members_dry_90patcor[year_range.index(year),m-1,:,:] = sst_seasonal_anom.s_anom[sst_selection]
            
            PREC_precip_selection = ((PREC_precip_ca.ymdhms[:, 0] == year) & (PREC_precip_ca.ymdhms[:, 1] == m))
            PREC_precip_selection = np.nonzero(PREC_precip_selection)[0][0]
            PREC_precip_composite_members_dry_90patcor[year_range.index(year),m-1,:,:] = PREC_precip_seasonal_anom.s_anom[PREC_precip_selection]
            
            selection = ((uwnd_ca.ymdhms[:, 0] == year) & (uwnd_ca.ymdhms[:, 1] == m))
            selection = np.nonzero(selection)[0][0]
            uwnd_composite_members_dry_90patcor[year_range.index(year),m-1,:,:] = uwnd_seasonal_anom.s_anom[selection]
            vwnd_composite_members_dry_90patcor[year_range.index(year),m-1,:,:] = vwnd_seasonal_anom.s_anom[selection]
            hgt_composite_members_dry_90patcor[year_range.index(year),m-1,:,:] = hgt_seasonal_anom.s_anom[selection]
            
            season_members_dry_90patcor[m-1] = season_members_dry_90patcor[m-1] +1
            
            precip_selection = ((precip_ca.ymdhms[:, 0] == year) & (precip_ca.ymdhms[:, 1] == m))
            precip_selection = np.nonzero(precip_selection)[0][0]
            precip_composite_members_dry_90patcor[year_range.index(year),m-1,:,:] = precip_seasonal_anom.s_anom[precip_selection]

            gpcp_season_members_dry_90patcor[m-1] = gpcp_season_members_dry_90patcor[m-1] +1



0 1981
4 1981
5 1981
6 1981
8 1981
0 1984
1 1984
2 1984
3 1984
4 1984
5 1984
6 1984
7 1984
8 1984
9 1984
10 1984
7 1985
8 1985
9 1985
10 1985
3 1988
4 1988
5 1988
6 1988
5 1989
6 1989
10 1989
5 1996
6 1996
7 1996
8 1996
9 1996
5 1998
11 1998
2 1999
3 1999
4 1999
7 1999
8 1999
3 2000
4 2000
5 2000
9 2000
1 2001
2 2001
3 2001
4 2001
5 2001
7 2001
8 2001
0 2006
2 2006
5 2007
6 2007
4 2008
6 2008
7 2008
8 2008
9 2008
10 2008
11 2008
0 2009
1 2009
2 2009
3 2009
5 2010
6 2010
7 2010
2 2012
1 2013
3 2013
4 2013
5 2013
6 2013
7 2013
8 2013
9 2013
10 2013


In [106]:
precip_composite_verydry = np.nanmean(precip_composite_members_verydry, axis=0)
sst_composite_verydry = np.nanmean(sst_composite_members_verydry, axis=0)
PREC_precip_composite_verydry = np.nanmean(PREC_precip_composite_members_verydry, axis=0)
uwnd_composite_verydry = np.nanmean(uwnd_composite_members_verydry, axis=0)
vwnd_composite_verydry = np.nanmean(vwnd_composite_members_verydry, axis=0)
hgt_composite_verydry = np.nanmean(hgt_composite_members_verydry, axis=0)

precip_composite_dry = np.nanmean(precip_composite_members_dry, axis=0)
sst_composite_dry = np.nanmean(sst_composite_members_dry, axis=0)
PREC_precip_composite_dry  = np.nanmean(PREC_precip_composite_members_dry, axis=0)
uwnd_composite_dry = np.nanmean(uwnd_composite_members_dry, axis=0)
vwnd_composite_dry = np.nanmean(vwnd_composite_members_dry, axis=0)
hgt_composite_dry = np.nanmean(hgt_composite_members_dry, axis=0)

In [107]:
precip_composite_verydry_90patcor = np.nanmean(precip_composite_members_verydry_90patcor, axis=0)
sst_composite_verydry_90patcor = np.nanmean(sst_composite_members_verydry_90patcor, axis=0)
PREC_precip_composite_verydry_90patcor = np.nanmean(PREC_precip_composite_members_verydry_90patcor, axis=0)
uwnd_composite_verydry_90patcor = np.nanmean(uwnd_composite_members_verydry_90patcor, axis=0)
vwnd_composite_verydry_90patcor = np.nanmean(vwnd_composite_members_verydry_90patcor, axis=0)
hgt_composite_verydry_90patcor = np.nanmean(hgt_composite_members_verydry_90patcor, axis=0)

precip_composite_dry_90patcor = np.nanmean(precip_composite_members_dry_90patcor, axis=0)
sst_composite_dry_90patcor = np.nanmean(sst_composite_members_dry_90patcor, axis=0)
PREC_precip_composite_dry_90patcor = np.nanmean(PREC_precip_composite_members_dry_90patcor, axis=0)
uwnd_composite_dry_90patcor = np.nanmean(uwnd_composite_members_dry_90patcor, axis=0)
vwnd_composite_dry_90patcor = np.nanmean(vwnd_composite_members_dry_90patcor, axis=0)
hgt_composite_dry_90patcor = np.nanmean(hgt_composite_members_dry_90patcor, axis=0)

/home/alejandro/anaconda3/lib/python3.5/site-packages/numpy/lib/nanfunctions.py:675: RuntimeWarning: Mean of empty slice
  warnings.warn("Mean of empty slice", RuntimeWarning)
/home/alejandro/anaconda3/lib/python3.5/site-packages/numpy/lib/nanfunctions.py:675: RuntimeWarning: Mean of empty slice
  warnings.warn("Mean of empty slice", RuntimeWarning)
/home/alejandro/anaconda3/lib/python3.5/site-packages/numpy/lib/nanfunctions.py:675: RuntimeWarning: Mean of empty slice
  warnings.warn("Mean of empty slice", RuntimeWarning)
/home/alejandro/anaconda3/lib/python3.5/site-packages/numpy/lib/nanfunctions.py:675: RuntimeWarning: Mean of empty slice
  warnings.warn("Mean of empty slice", RuntimeWarning)
/home/alejandro/anaconda3/lib/python3.5/site-packages/numpy/lib/nanfunctions.py:675: RuntimeWarning: Mean of empty slice
  warnings.warn("Mean of empty slice", RuntimeWarning)
/home/alejandro/anaconda3/lib/python3.5/site-packages/numpy/lib/nanfunctions.py:675: RuntimeWarning: Mean of empty slice

In [108]:
season_list = ['DJF' , 'JFM' , 'FMA',
               'MAM' , 'AMJ' , 'MJJ',
               'JJA' , 'JAS' , 'ASO',
               'SON' , 'OND' , 'NDJ']

month_list = ['Jan' , 'Feb' , 'Mar',
              'Apr' , 'May' , 'Jun',
              'Jul' , 'Aug' , 'Sep',
              'Oct' , 'Nov' , 'Dec']


plt.rcParams['figure.dpi'] = 113
plt.rcParams['axes.titlesize'] = 'small'
plt.rcParams['ytick.labelsize'] = 'small'  # for colorbar

# Here I define the plotting function

In [120]:
def plotting_function(color_field, contour_field, u_wnd, v_wnd, member_counter, 
                      color_field_name, case_name, *argv, save = 'yes'):    
    
    #here basemap is producing a strange behaviour
    m = Basemap(projection='merc', llcrnrlat=-50.1, urcrnrlat=50.01,
                        llcrnrlon=80, urcrnrlon=300, lat_ts=0, resolution='c')

    with open('mapcache.pk', mode='wb') as f:
        pickle.dump(m, f)

    ## Tweak the subplot specifications.  

    subplotparams = dict(left=0.03, right=0.88,
                         bottom=0.015, top=0.94,
                         wspace=0.05, hspace=0.001)


    fig, axs = plt.subplots(3,3, #sharex=True,
                        figsize=(13, 7.8),
                        gridspec_kw=subplotparams,
                       )

    mm = range(2,11)

    for mon, pax in zip(mm, axs.flat):

        with open('mapcache.pk', 'rb') as f:
            m = pickle.load(f)
        m.ax = pax

        #reverse the coolwarm palatte to make blue rainy and red dry
        cmap = plt.get_cmap('BrBG')
        bounds = [-2.5,-2, -1.5,-1.25, -1, -0.75, -0.5, -0.25, 0, 0.25, 0.5, 0.75, 1, 1.25, 1.5, 2, 2.5]


        x, y = m(*np.meshgrid(PREC_precip_longitudes, PREC_precip_latitudes))
        im = m.contourf(x, y, color_field[mon,:,:], levels=bounds, cmap=cmap,
                        extend='both')

        x_hgt, y_hgt = m(*np.meshgrid(ncep_longitudes, ncep_latitudes))
        #hgt_bounds = np.arange(-40,40,1)
        hgt_bounds = [-80, -70,-60, -50, -40, 
                      -30, -20, -10,-8, -6, 
                      -4, -2, -1,0, 1, 2, 4, 
                      6, 8, 10, 20, 30,
                      40, 50, 60, 70, 80]
        hgt_bound_0 = [0]
        im_hgt = m.contour(x_hgt, y_hgt, contour_field[mon,:,:], levels=hgt_bounds,
                           linewidths=0.5, colors='k')
        im_hgt = m.contour(x_hgt, y_hgt, contour_field[mon,:,:], levels=hgt_bound_0,
                       linewidths=0.9, colors='k')


        # transform vectors to projection grid.
        uproj, vproj, xx, yy = m.rotate_vector(u_wnd[mon,:,:], 
                                               v_wnd[mon,:,:], 
                                               ncep_longitudes, 
                                               ncep_latitudes,
                                               returnxy=True)
        # now plot every other vector
        Q = m.quiver(xx[::2,::2], yy[::2,::2], 
                     uproj[::2,::2], vproj[::2,::2],
                     scale=20, scale_units='inches')

        m.drawcoastlines()
        
        if box_coords in argv:
            x1,y1 = m(box_coords[2], box_coords[1])
            x2,y2 = m(box_coords[2], box_coords[0])
            x3,y3 = m(box_coords[3], box_coords[0])
            x4,y4 = m(box_coords[3], box_coords[1])
            
            p = Polygon([(x1,y1), (x2,y2), (x3,y3), (x4,y4)], 
                        edgecolor='b', facecolor = 'none', linewidth = 2)
            m.ax.add_patch(p)
        
        parallels = np.arange(-90, 90, 30)
        meridians = np.arange(-180, 180, 60)
        if pax in axs.flat[::3]:
            m.drawparallels(parallels, labels = [1, 0, 0, 1], fontsize=8)
        else:
            m.drawparallels(parallels, labels = [0, 0, 0, 0], fontsize=8)

        if pax in axs.flat[:6]:
            m.drawmeridians(meridians, labels = [0, 0, 0, 0], fontsize=8)
        if pax in axs.flat[6:]:
            m.drawmeridians(meridians, labels = [1, 0, 0, 1], fontsize=8)

        #m.drawparallels(parallels, labels = [1, 0, 0, 1], fontsize=8)
        #m.drawmeridians(meridians, labels = [1, 0, 0, 1], fontsize=8)

        pax.set_title(color_field_name+" "+season_list[mon]+ " Anom Comp. Memb " + str(member_counter[mon]))

    #cax, kw = mpl.colorbar.make_axes([ax for ax in axs.flat])
    ##  This method of making an Axes for the cbar interacts very badly with
    ##  basemap, so just make one manually.  This also gives more control, so
    ##  it looks nicer.
    left = subplotparams['right'] + 0.02
    bottom = subplotparams['bottom'] + 0.05
    width = 0.015
    height = subplotparams['top'] - subplotparams['bottom'] - 0.1

    cax = fig.add_axes([left, bottom, width, height])
    cb = plt.colorbar(im, cax=cax)
    cb.set_label('Precip Anomaly mm/day')

    ## EF: The quiverkey needs to be made using an axes in which a quiver is
    ##     drawn so that it has the right transform information.  Otherwise
    ##     it won't get the length right.
    qkx = left + (1 - left) / 4
    qky = bottom + height + 0.025
    qk = pax.quiverkey(Q, qkx, qky, 10, '10 m/s', 
                       coordinates='figure',
                       labelpos='N',
                       labelsep=0.07, 
                       fontproperties=dict(size='small'),
                      )
    
    plt.suptitle(case_name+" "+color_field_name+" PRECIP 850HGT WND seasonal anomaly composite. 1979-2015 clim")
    
    if save == 'yes':
        fig.savefig("./clim_1979-2015/"+case_name+"_"+color_field_name+"_PRECIP_850HGT_WND_seasonal_comp_79_15_clim.pdf")
        fig.savefig("./clim_1979-2015/"+case_name+"_"+color_field_name+"_PRECIP_850HGT_WND_seasonal_comp_79_15_clim.png")
    #fig.savefig("VERY_DRY_COMPOSITE_SST_850HGT_WND_seasonal_anomaly_%04d-%02d.pdf" % (year, mon))
        plt.close()# no need for dpi

In [121]:
def sst_plotting_function(color_field, contour_field, u_wnd, v_wnd, member_counter, 
                          color_field_name, case_name, *argv, save = 'yes'):    
    
    #here basemap is producing a strange behaviour
    m = Basemap(projection='merc', llcrnrlat=-50.1, urcrnrlat=50.01,
                        llcrnrlon=80, urcrnrlon=300, lat_ts=0, resolution='c')

    with open('mapcache.pk', mode='wb') as f:
        pickle.dump(m, f)

    ## Tweak the subplot specifications.  

    subplotparams = dict(left=0.03, right=0.88,
                         bottom=0.015, top=0.94,
                         wspace=0.05, hspace=0.001)


    fig, axs = plt.subplots(3,3, #sharex=True,
                        figsize=(13, 7.8),
                        gridspec_kw=subplotparams,
                       )

    mm = range(2,11)

    for mon, pax in zip(mm, axs.flat):

        with open('mapcache.pk', 'rb') as f:
            m = pickle.load(f)
        m.ax = pax

        #reverse the coolwarm palatte to make blue rainy and red dry
        cmap = cm.coolwarm
        bounds = [-2, -1.5,-1.25, -1, -0.75, -0.5, -0.25, 0, 0.25, 0.5, 0.75, 1, 1.25, 1.5, 2]


        x, y = m(*np.meshgrid(ersst_longitudes, ersst_latitudes))
        im = m.contourf(x, y, color_field[mon,:,:], levels=bounds, cmap=cmap,
                        extend='both')

        x_hgt, y_hgt = m(*np.meshgrid(ncep_longitudes, ncep_latitudes))
        #hgt_bounds = np.arange(-40,40,1)
        hgt_bounds = [-80, -70,-60, -50, -40, 
                      -30, -20, -10,-8, -6, 
                      -4, -2, -1,0, 1, 2, 4, 
                      6, 8, 10, 20, 30,
                      40, 50, 60, 70, 80]
        hgt_bound_0 = [0]
        im_hgt = m.contour(x_hgt, y_hgt, contour_field[mon,:,:], levels=hgt_bounds,
                           linewidths=0.5, colors='k')
        im_hgt = m.contour(x_hgt, y_hgt, contour_field[mon,:,:], levels=hgt_bound_0,
                       linewidths=0.9, colors='k')


        # transform vectors to projection grid.
        uproj, vproj, xx, yy = m.rotate_vector(u_wnd[mon,:,:], 
                                               v_wnd[mon,:,:], 
                                               ncep_longitudes, 
                                               ncep_latitudes,
                                               returnxy=True)
        # now plot every other vector
        Q = m.quiver(xx[::2,::2], yy[::2,::2], 
                     uproj[::2,::2], vproj[::2,::2],
                     scale=20, scale_units='inches')

        m.drawcoastlines()
        
        #draw the box
        #lons = range(180,200,2.5)
        #lats = 
        #m.drawgreatcircle(180,20,180,0,linewidth = 2, color = 'b')
        
        if box_coords in argv:
            x1,y1 = m(box_coords[2], box_coords[1])
            x2,y2 = m(box_coords[2], box_coords[0])
            x3,y3 = m(box_coords[3], box_coords[0])
            x4,y4 = m(box_coords[3], box_coords[1])
            
            p = Polygon([(x1,y1), (x2,y2), (x3,y3), (x4,y4)], 
                        edgecolor='b', facecolor = 'none', linewidth = 2)
            m.ax.add_patch(p)
            #plt.gca().add_patch(p)

        
        parallels = np.arange(-90, 90, 30)
        meridians = np.arange(-180, 180, 60)
        if pax in axs.flat[::3]:
            m.drawparallels(parallels, labels = [1, 0, 0, 1], fontsize=8)
        else:
            m.drawparallels(parallels, labels = [0, 0, 0, 0], fontsize=8)

        if pax in axs.flat[:6]:
            m.drawmeridians(meridians, labels = [0, 0, 0, 0], fontsize=8)
        if pax in axs.flat[6:]:
            m.drawmeridians(meridians, labels = [1, 0, 0, 1], fontsize=8)

        #m.drawparallels(parallels, labels = [1, 0, 0, 1], fontsize=8)
        #m.drawmeridians(meridians, labels = [1, 0, 0, 1], fontsize=8)

        pax.set_title(color_field_name+" "+season_list[mon]+ " Anom Comp. Memb " + str(member_counter[mon]))

    #cax, kw = mpl.colorbar.make_axes([ax for ax in axs.flat])
    ##  This method of making an Axes for the cbar interacts very badly with
    ##  basemap, so just make one manually.  This also gives more control, so
    ##  it looks nicer.
    left = subplotparams['right'] + 0.02
    bottom = subplotparams['bottom'] + 0.05
    width = 0.015
    height = subplotparams['top'] - subplotparams['bottom'] - 0.1

    cax = fig.add_axes([left, bottom, width, height])
    cb = plt.colorbar(im, cax=cax)
    cb.set_label('SST Anomaly Deg C')

    ## EF: The quiverkey needs to be made using an axes in which a quiver is
    ##     drawn so that it has the right transform information.  Otherwise
    ##     it won't get the length right.
    qkx = left + (1 - left) / 4
    qky = bottom + height + 0.025
    qk = pax.quiverkey(Q, qkx, qky, 10, '10 m/s', 
                       coordinates='figure',
                       labelpos='N',
                       labelsep=0.07, 
                       fontproperties=dict(size='small'),
                      )
    
    plt.suptitle(case_name+" "+color_field_name+" SST 850HGT WND seasonal anomaly composite. 1979-2015 clim")
    
    if save == 'yes':
        fig.savefig("./clim_1979-2015/"+case_name+"_"+color_field_name+"_SST_850HGT_WND_seasonal_comp_79_15_clim.pdf")
        fig.savefig("./clim_1979-2015/"+case_name+"_"+color_field_name+"_SST_850HGT_WND_seasonal_comp_79_15_clim.png")
        #fig.savefig("VERY_DRY_COMPOSITE_SST_850HGT_WND_seasonal_anomaly_%04d-%02d.pdf" % (year, mon))
        plt.close()# no need for dpi

In [122]:
#plotting_function(PREC_precip_composite_verydry, hgt_composite_verydry, 
#                  uwnd_composite_verydry, vwnd_composite_verydry, 
#                  season_members_verydry, 'PREC', "Very_Dry")

box_coords =[]

# Here we plot the composites for the verydry cases

## verydry case composite with PREC data

In [130]:
plotting_function(PREC_precip_composite_verydry_90patcor, hgt_composite_verydry_90patcor, 
                  uwnd_composite_verydry_90patcor, vwnd_composite_verydry_90patcor, 
                  season_members_verydry_90patcor, 'PREC', "Very_Dry_90patcor",  save = 'yes')

plotting_function(PREC_precip_composite_dry, hgt_composite_dry, 
                  uwnd_composite_dry, vwnd_composite_dry, 
                  season_members_dry, 'PREC', "Dry",  save = 'yes')

## GPCP verydry 90 patcor composites

In [131]:
plotting_function(precip_composite_verydry, hgt_composite_verydry, 
                  uwnd_composite_verydry, vwnd_composite_verydry, 
                  gpcp_season_members_verydry, 'GPCP', "Very_Dry",  save = 'yes')


plotting_function(precip_composite_verydry_90patcor, hgt_composite_verydry_90patcor, 
                  uwnd_composite_verydry_90patcor, vwnd_composite_verydry_90patcor, 
                  gpcp_season_members_verydry_90patcor, 'GPCP', "Very_Dry_90patcor",  save = 'yes')

plotting_function(precip_composite_dry, hgt_composite_dry, 
                  uwnd_composite_dry, vwnd_composite_dry, 
                  gpcp_season_members_dry, 'GPCP', "Dry",  save = 'yes')


plotting_function(precip_composite_dry_90patcor, hgt_composite_dry_90patcor, 
                  uwnd_composite_dry_90patcor, vwnd_composite_dry_90patcor, 
                  gpcp_season_members_dry_90patcor, 'GPCP', "Dry_90patcor",  save = 'yes')

## SST maps

In [132]:
sst_plotting_function(sst_composite_verydry, hgt_composite_verydry, 
                      uwnd_composite_verydry, vwnd_composite_verydry, 
                      season_members_verydry, 'ERSST', "Very_Dry",  save = 'yes')


sst_plotting_function(sst_composite_verydry_90patcor, hgt_composite_verydry_90patcor, 
                      uwnd_composite_verydry_90patcor, vwnd_composite_verydry_90patcor, 
                      season_members_verydry_90patcor, 'ERSST', "Very_Dry_90patcor",  save = 'yes')

sst_plotting_function(sst_composite_dry, hgt_composite_dry, 
                      uwnd_composite_dry, vwnd_composite_dry, 
                      season_members_dry, 'ERSST', "Dry",  save = 'yes')


sst_plotting_function(sst_composite_dry_90patcor, hgt_composite_dry_90patcor, 
                      uwnd_composite_dry_90patcor, vwnd_composite_dry_90patcor, 
                      season_members_dry_90patcor, 'ERSST', "Dry_90patcor",  save = 'yes')

# Make the non dry composites

In [126]:
cond_non_dry = kwajalein_seasonal_anom_matrix >= 0* kwajalein_season_std_matrix
cond_oni = oni_period_matrix <= -0.1
cond_non_dry_cool = np.logical_and(cond_non_dry, cond_oni)

precip_composite_members_non_dry = np.empty([50,12, len(precip_latitudes), len(precip_longitudes)])
precip_composite_members_non_dry[:] = np.NAN

sst_composite_members_non_dry = np.empty([50,12, len(ersst_latitudes), len(ersst_longitudes)])
sst_composite_members_non_dry[:] = np.NAN

PREC_precip_composite_members_non_dry = np.empty([50,12, len(PREC_precip_latitudes), len(PREC_precip_longitudes)])
PREC_precip_composite_members_non_dry[:] = np.NAN

uwnd_non_dry_composite_members = np.empty([50,12,len(ncep_latitudes), len(ncep_longitudes)])
uwnd_non_dry_composite_members[:] = np.NAN
vwnd_non_dry_composite_members = np.empty([50,12,len(ncep_latitudes), len(ncep_longitudes)])
vwnd_non_dry_composite_members[:] = np.NAN
hgt_non_dry_composite_members = np.empty([50,12,len(ncep_latitudes), len(ncep_longitudes)])
hgt_non_dry_composite_members[:] = np.NAN

gpcp_season_members_non_dry = np.zeros(12)
season_members_non_dry = np.zeros(12)

In [127]:
year_range = range(1966,2016)
short_year_range = range(1979,2016)
for year in short_year_range:
    for m in range(1,13):
        
        if cond_non_dry_cool[year_range.index(year),m-1]:
            
            sst_selection = ((sst_ca.ymdhms[:, 0] == year) & (sst_ca.ymdhms[:, 1] == m))
            sst_selection = np.nonzero(sst_selection)[0][0]
            sst_composite_members_non_dry[year_range.index(year),m-1,:,:] = sst_seasonal_anom.s_anom[sst_selection]
            
            PREC_precip_selection = ((PREC_precip_ca.ymdhms[:, 0] == year) & (PREC_precip_ca.ymdhms[:, 1] == m))
            PREC_precip_selection = np.nonzero(PREC_precip_selection)[0][0]
            PREC_precip_composite_members_non_dry[year_range.index(year),m-1,:,:] = PREC_precip_seasonal_anom.s_anom[PREC_precip_selection]
            
            selection = ((uwnd_ca.ymdhms[:, 0] == year) & (uwnd_ca.ymdhms[:, 1] == m))
            selection = np.nonzero(selection)[0][0]
            uwnd_non_dry_composite_members[year_range.index(year),m-1,:,:] = uwnd_seasonal_anom.s_anom[selection]
            vwnd_non_dry_composite_members[year_range.index(year),m-1,:,:] = vwnd_seasonal_anom.s_anom[selection]
            hgt_non_dry_composite_members[year_range.index(year),m-1,:,:] = hgt_seasonal_anom.s_anom[selection]
            
            season_members_non_dry[m-1] = season_members_non_dry[m-1] +1
            

            precip_selection = ((precip_ca.ymdhms[:, 0] == year) & (precip_ca.ymdhms[:, 1] == m))
            precip_selection = np.nonzero(precip_selection)[0][0]
            precip_composite_members_non_dry[year_range.index(year),m-1,:,:] = precip_seasonal_anom.s_anom[precip_selection]

            gpcp_season_members_non_dry[m-1] = gpcp_season_members_non_dry[m-1] +1

In [128]:
precip_non_dry_composite = np.nanmean(precip_composite_members_non_dry, axis=0)

sst_non_dry_composite = np.nanmean(sst_composite_members_non_dry, axis=0)

PREC_precip_non_dry_composite = np.nanmean(PREC_precip_composite_members_non_dry, axis=0)

uwnd_non_dry_composite = np.nanmean(uwnd_non_dry_composite_members, axis=0)
vwnd_non_dry_composite = np.nanmean(vwnd_non_dry_composite_members, axis=0)
hgt_non_dry_composite = np.nanmean(hgt_non_dry_composite_members, axis=0)

In [133]:
plotting_function(PREC_precip_non_dry_composite, hgt_non_dry_composite, 
                  uwnd_non_dry_composite, vwnd_non_dry_composite, 
                  season_members_non_dry, 'PREC', "Non_Dry",  save = 'yes')

plotting_function(precip_non_dry_composite, hgt_non_dry_composite, 
                  uwnd_non_dry_composite, vwnd_non_dry_composite, 
                  gpcp_season_members_non_dry, 'GPCP', "Non_Dry",  save = 'yes')

sst_plotting_function(sst_non_dry_composite, hgt_non_dry_composite, 
                      uwnd_non_dry_composite, vwnd_non_dry_composite, 
                      season_members_non_dry, 'ERSST', "Non_Dry",  save = 'yes')

# Stop Here-------------------------^

In [41]:
#here basemap is producing a strange behaviour
m = Basemap(projection='merc', llcrnrlat=-50.1, urcrnrlat=50.01,
                    llcrnrlon=80, urcrnrlon=300, lat_ts=0, resolution='c')

with open('mapcache.pk', mode='wb') as f:
    pickle.dump(m, f)

## Tweak the subplot specifications.  

subplotparams = dict(left=0.03, right=0.88,
                     bottom=0.03, top=0.96,
                     wspace=0.05, hspace=0.1)

    
fig, axs = plt.subplots(3,3, #sharex=True,
                    figsize=(13, 7.8),
                    gridspec_kw=subplotparams,
                   )

mm = range(2,11)

for mon, pax in zip(mm, axs.flat):

    with open('mapcache.pk', 'rb') as f:
        m = pickle.load(f)
    m.ax = pax

    #reverse the coolwarm palatte to make blue rainy and red dry
    cmap = cm.coolwarm
    bounds = [-2, -1.5,-1.25, -1, -0.75, -0.5, -0.25, 0, 0.25, 0.5, 0.75, 1, 1.25, 1.5, 2]
    #norm = cols.BoundaryNorm(bounds, cmap.N)

    #im = m.imshow(sst_transform,cmap=cm.coolwarm, interpolation = "none",norm=norm)

    ##  EF: using an image gains nothing here, loses resolution, and adds complexity.
    ##      Contouring is nicer.  If you wanted something image-like, it would
    ##      be better to use pcolormesh--but there is no point when you are using
    ##      discretized colors on a relatively fine lon/lat grid like this.

    x, y = m(*np.meshgrid(ersst_longitudes, ersst_latitudes))
    im = m.contourf(x, y, sst_non_dry_composite[mon,:,:], levels=bounds, cmap=cmap,
                    extend='both')

    x_hgt, y_hgt = m(*np.meshgrid(ncep_longitudes, ncep_latitudes))
    #hgt_bounds = np.arange(-40,40,1)
    hgt_bounds = [-80, -70,-60, -50, -40, 
                  -30, -20, -10,-8, -6, 
                  -4, -2, -1,0, 1, 2, 4, 
                  6, 8, 10, 20, 30,
                  40, 50, 60, 70, 80]
    im_hgt = m.contour(x_hgt, y_hgt, hgt_non_dry_composite[mon,:,:], levels=hgt_bounds,
                       linewidths=0.5, colors='k')


    # transform vectors to projection grid.
    uproj, vproj, xx, yy = m.rotate_vector(uwnd_non_dry_composite[mon,:,:], 
                                           vwnd_non_dry_composite[mon,:,:], 
                                           ncep_longitudes, 
                                           ncep_latitudes,
                                           returnxy=True)
    # now plot every other vector
    Q = m.quiver(xx[::2,::2], yy[::2,::2], 
                 uproj[::2,::2], vproj[::2,::2],
                 scale=20, scale_units='inches')

    m.drawcoastlines()
    parallels = np.arange(-90, 90, 30)
    meridians = np.arange(-180, 180, 60)
    if pax in axs.flat[::3]:
        m.drawparallels(parallels, labels = [1, 0, 0, 1], fontsize=8)
    else:
        m.drawparallels(parallels, labels = [0, 0, 0, 0], fontsize=8)

    if pax in axs.flat[:6]:
        m.drawmeridians(meridians, labels = [0, 0, 0, 0], fontsize=8)
    if pax in axs.flat[6:]:
        m.drawmeridians(meridians, labels = [1, 0, 0, 1], fontsize=8)

    #m.drawparallels(parallels, labels = [1, 0, 0, 1], fontsize=8)
    #m.drawmeridians(meridians, labels = [1, 0, 0, 1], fontsize=8)

    pax.set_title("Seasonal Anomaly Composite " +season_list[mon] + " members " + str(season_members_non_dry[mon]))

#cax, kw = mpl.colorbar.make_axes([ax for ax in axs.flat])
##  This method of making an Axes for the cbar interacts very badly with
##  basemap, so just make one manually.  This also gives more control, so
##  it looks nicer.
left = subplotparams['right'] + 0.02
bottom = subplotparams['bottom'] + 0.05
width = 0.015
height = subplotparams['top'] - subplotparams['bottom'] - 0.1

cax = fig.add_axes([left, bottom, width, height])
cb = plt.colorbar(im, cax=cax)
cb.set_label('SST C')

## EF: The quiverkey needs to be made using an axes in which a quiver is
##     drawn so that it has the right transform information.  Otherwise
##     it won't get the length right.
qkx = left + (1 - left) / 4
qky = bottom + height + 0.025
qk = pax.quiverkey(Q, qkx, qky, 10, '10 m/s', 
                   coordinates='figure',
                   labelpos='N',
                   labelsep=0.07, 
                   fontproperties=dict(size='small'),
                  )

fig.savefig("NON_DRY_COMPOSITE_SST_850HGT_WNDseasonal_anomaly.pdf")
#plt.close()# no need for dpi 